In [56]:
import geopy.distance
from geopy.distance import geodesic
import s3fs
import xarray as xr

import os.path
import os
import sys

from datetime import datetime, timedelta
import dataclasses
from siphon.cdmr import Dataset
import numpy as np
import numpy.ma as ma
import netCDF4

from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

from metpy.calc import (bunkers_storm_motion, bulk_shear, dewpoint, dewpoint_from_relative_humidity, dry_lapse, moist_lapse,
                        vapor_pressure, saturation_vapor_pressure, wind_speed, wind_direction, pressure_to_height_std,
                        mixing_ratio, cape_cin, wind_components, height_to_pressure_std, equivalent_potential_temperature,
                        parcel_profile, precipitable_water, storm_relative_helicity, mean_pressure_weighted, 
                        most_unstable_cape_cin, most_unstable_parcel, supercell_composite, significant_tornado, get_layer,
                        relative_humidity_from_dewpoint, surface_based_cape_cin, mixed_layer_cape_cin,
                        surface_based_cape_cin, potential_temperature, wind_direction, add_pressure_to_height,
                        add_height_to_pressure, divergence, vorticity, lat_lon_grid_deltas, mixed_parcel,
                        most_unstable_parcel, lcl, lfc, mixing_ratio_from_relative_humidity, el,  cape_cin,
                        height_to_pressure_std, dewpoint_from_relative_humidity, dewpoint_from_specific_humidity,
                        relative_humidity_from_specific_humidity, first_derivative)
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import SkewT, Hodograph
from metpy.interpolate import interpolate_1d as metinterp, log_interpolate_1d
from metpy.calc.tools import get_layer, get_layer_heights

import sharppy
import sharppy.sharptab.profile as profile
import sharppy.sharptab.interp as interp
import sharppy.sharptab.winds as winds
import sharppy.sharptab.utils as utils
import sharppy.sharptab.params as params
import sharppy.sharptab.thermo as thermo

from ecape.calc import calc_ecape

import csv
import pandas as pd
import math
import time

In [57]:
#https://citylikeamradio.github.io/ecape/html/index.html
# SPDX-FileCopyrightText: 2023-present Robert Capella <bob.capella@gmail.com>
# SPDX-License-Identifier: MIT
"""Calculate the entraining CAPE (ECAPE) of a parcel"""
from typing import Callable, Tuple

import pint
from metpy.constants import dry_air_spec_heat_press, earth_gravity
from metpy.units import check_units, units

PintList = np.typing.NDArray[pint.Quantity]

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

@check_units("[pressure]", "[temperature]", "[temperature]")
def _get_parcel_profile(
    pressure: PintList, temperature: PintList, dew_point_temperature: PintList, parcel_func: Callable = None
) -> PintList:
    """
    Retrieve a parcel's temperature profile.

    Args:
        pressure:
            Total atmospheric pressure
        temperature:
            Air temperature
        dew_point_temperature:
            Dew point temperature
        parcel_func:
            parcel profile retrieval callable via MetPy

    Returns:
        parcel_profile

    """

    # if surface-based, skip this process, None is default for lfc, el MetPy calcs
    if parcel_func:
        # calculate the parcel's starting temperature, then parcel temperature profile
        parcel_p, parcel_t, parcel_td, *parcel_i = parcel_func(pressure, temperature, dew_point_temperature)
        parcel_profile = mpcalc.parcel_profile(pressure, parcel_t, parcel_td)
    else:
        parcel_profile = None

    return parcel_profile

@check_units("[pressure]", "[length]", "[temperature]", "[temperature]")
def calc_lfc_height(
    pressure: PintList, height_msl: PintList, temperature: PintList, dew_point_temperature: PintList, parcel_func: Callable
) -> Tuple[int, pint.Quantity]:
    """
    Retrieve a parcel's level of free convection (lfc).

    Args:
        pressure:
            Total atmospheric pressure
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        temperature:
            Air temperature
        dew_point_temperature:
            Dew point temperature
        parcel_func:
            parcel profile retrieval callable via MetPy

    Returns:
        lfc:
            index of the last instance of negative buoyancy below the lfc
        lfc_z:
            height of the last instance of negative buoyancy below the lfc

    """

    # calculate the parcel's temperature profile
    parcel_profile = _get_parcel_profile(pressure, temperature, dew_point_temperature, parcel_func)

    # calculate the lfc, select the appropriate index & associated height
    lfc_p, lfc_t = mpcalc.lfc(pressure, temperature, dew_point_temperature, parcel_temperature_profile=parcel_profile)
    
    #press = np.round(np.asarray(P_sounding),8)
    nearestLFC=find_nearest(np.asarray(pressure),lfc_p.magnitude)
    lfc_idx = np.where(np.asarray(pressure)==nearestLFC)
    lfc_idx=lfc_idx[0][0]
    
    #lfc_idx = (pressure - lfc_p > 0).nonzero()[0][-1]
    lfc_z = height_msl[lfc_idx]

    return lfc_idx, lfc_z

@check_units("[pressure]", "[length]", "[temperature]", "[temperature]")
def calc_el_height(
    pressure: PintList, height_msl: PintList, temperature: PintList, dew_point_temperature: PintList, parcel_func: Callable
) -> Tuple[int, pint.Quantity]:
    """
    Retrieve a parcel's equilibrium level (el).

    Args:
        pressure:
            Total atmospheric pressure
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        temperature:
            Air temperature
        dew_point_temperature:
            Dew point temperature
        parcel_func:
            parcel profile retrieval callable via MetPy

    Returns:
        el_idx:
            index of the last instance of positive buoyancy below the el
        el_z:
            height of the last instance of positive buoyancy below the el

    """

    # calculate the parcel's temperature profile
    parcel_profile = _get_parcel_profile(pressure, temperature, dew_point_temperature, parcel_func)

    # calculate the el, select the appropriate index & associated height
    el_p, el_t = mpcalc.el(pressure, temperature, dew_point_temperature, parcel_temperature_profile=parcel_profile)
    
    #press = np.round(np.asarray(P_sounding),8)
    #nearestEL=(find_nearest(press,el_p.magnitude))
    #el_idx = np.where(press==nearestEL)
    #el_idx=el_idx[0][0]
    
    nearestEL=find_nearest(np.asarray(pressure),el_p.magnitude)
    el_idx = np.where(np.asarray(pressure)==nearestEL)
    el_idx=el_idx[0][0]
    
    #el_idx = (pressure - el_p > 0).nonzero()[0][-1]
    el_z = height_msl[el_idx]

    return el_idx, el_z

@check_units("[pressure]", "[speed]", "[speed]", "[length]")
def calc_sr_wind(pressure: PintList, u_wind: PintList, v_wind: PintList, height_msl: PintList) -> pint.Quantity:
    """
    Calculate the mean storm relative (as compared to Bunkers right motion) wind magnitude in the 0-1 km AGL layer

    Args:
        pressure:
            Total atmospheric pressure
        u_wind:
            X component of the wind
        v_wind
            Y component of the wind
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.

    Returns:
        sr_wind:
            0-1 km AGL average storm relative wind magnitude

    """
    height_agl = height_msl - height_msl[0]
    bunkers_right, _, _ = mpcalc.bunkers_storm_motion(pressure, u_wind, v_wind, height_agl)  # right, left, mean

    u_sr = u_wind - bunkers_right[0]  # u-component
    v_sr = v_wind - bunkers_right[1]  # v-component

    u_sr_1km = u_sr[np.nonzero((height_agl >= 0 * units("m")) & (height_agl <= 1000 * units("m")))]
    v_sr_1km = v_sr[np.nonzero((height_agl >= 0 * units("m")) & (height_agl <= 1000 * units("m")))]

    sr_wind = np.mean(mpcalc.wind_speed(u_sr_1km, v_sr_1km))

    return sr_wind

@check_units("[pressure]", "[length]", "[temperature]", "[mass]/[mass]")
def calc_mse(
    pressure: PintList, height_msl: PintList, temperature: PintList, specific_humidity: PintList
) -> Tuple[PintList, PintList]:
    """
    Calculate the moist static energy terms of interest.

    Args:
        pressure:
            Total atmospheric pressure
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        temperature:
            Air temperature
        specific_humidity:
            Specific humidity

    Returns:
        moist_static_energy_bar:
            Mean moist static energy from the surface to a layer
        moist_static_energy_star:
            Saturated moist static energy
    """

    # calculate MSE_bar
    moist_static_energy = mpcalc.moist_static_energy(height_msl, temperature, specific_humidity)
    moist_static_energy_bar = np.cumsum(moist_static_energy) / np.arange(1, len(moist_static_energy) + 1)
    moist_static_energy_bar = moist_static_energy_bar.to("J/kg")

    # calculate MSE*
    saturation_mixing_ratio = mpcalc.saturation_mixing_ratio(pressure, temperature)
    moist_static_energy_star = mpcalc.moist_static_energy(height_msl, temperature, saturation_mixing_ratio)
    moist_static_energy_star = moist_static_energy_star.to("J/kg")

    return moist_static_energy_bar, moist_static_energy_star

@check_units("[energy]/[mass]", "[energy]/[mass]", "[temperature]")
def calc_integral_arg(moist_static_energy_bar, moist_static_energy_star, temperature) -> PintList:
    """
    Calculate the contents of the integral defined in the NCAPE equation (54).

    Args:
        moist_static_energy_bar:
            Mean moist static energy from the surface to a layer
        moist_static_energy_star:
            Saturated moist static energy
        temperature:
            Air temperature

    Returns:
        integral_arg:
            Contents of integral defined in NCAPE eqn. 54

    """

    # NCAPE eqn 54 integrand, see compute_NCAPE.m L32
    integral_arg = -(earth_gravity / (dry_air_spec_heat_press * temperature)) * (
        moist_static_energy_bar - moist_static_energy_star
    )

    return integral_arg

@check_units("[length]/[time]**2", "[length]", "[dimensionless]", "[dimensionless]")
def calc_ncape(integral_arg: PintList, height_msl: PintList, lfc_idx: int, el_idx: int) -> pint.Quantity:
    """
    Calculate the buoyancy dilution potential (NCAPE)

    Args:
        integral_arg:
            Contents of integral defined in NCAPE eqn. 54
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        lfc_idx:
            Index of the last instance of negative buoyancy below the lfc
        el_idx:
            Index of the last instance of positive buoyancy below the el

    Returns:
        ncape:
            Buoyancy dilution potential of the free troposphere (eqn. 54)
    """

    # see compute_NCAPE.m L41
    ncape = np.sum(
        (0.5 * integral_arg[lfc_idx:el_idx] + 0.5 * integral_arg[lfc_idx + 1 : el_idx + 1])
        * (height_msl[lfc_idx + 1 : el_idx + 1] - height_msl[lfc_idx:el_idx])
    )

    return ncape

@check_units("[speed]", "[dimensionless]", "[length]**2/[time]**2", "[energy]/[mass]")
def calc_ecape_a(sr_wind: PintList, psi: pint.Quantity, ncape: pint.Quantity, cape: pint.Quantity) -> pint.Quantity:
    """
    Calculate the entraining cape of a parcel

    Args:
        sr_wind:
            0-1 km AGL average storm relative wind magnitude
        psi:
            Parameter defined in eqn. 52, constant for a given equilibrium level
        ncape:
            Buoyancy dilution potential of the free troposphere (eqn. 54)
        cape:
            Convective available potential energy (CAPE, user-defined type)
    Returns:
        ecape:
            Entraining CAPE (eqn. 55)
    """

    # broken into terms for readability
    term_a = sr_wind**2 / 2.0
    term_b = (-1 - psi - (2 * psi / sr_wind**2) * ncape) / (4 * psi / sr_wind**2)
    term_c = (
        np.sqrt((1 + psi + (2 * psi / sr_wind**2) * ncape) ** 2 + 8 * (psi / sr_wind**2) * (cape - (psi * ncape)))
    ) / (4 * psi / sr_wind**2)

    ecape_a = term_a + term_b + term_c

    # set to 0 if negative
    return ecape_a.to("J/kg") if ecape_a >= 0 else 0

@check_units("[length]")
def calc_psi(el_z: pint.Quantity) -> pint.Quantity:
    """
    Calculate the constant psi as denoted in eqn. 52

    Args:
        el_z:
            height of the last instance of positive buoyancy below the el

    Returns:
        psi:
            Parameter defined in eqn. 52, constant for a given equilibrium level, see COMPUTE_ECAPE.m L88 (pitchfork)
    """

    # additional constants as denoted in section 4 step 1.
    sigma = 1.6 * units("dimensionless")
    alpha = 0.8 * units("dimensionless")
    l_mix = 120.0 * units("m")
    pr = (1.0 / 3.0) * units("dimensionless")  # prandtl number
    ksq = 0.18 * units("dimensionless")  # von karman constant

    psi = (ksq * alpha**2 * np.pi**2 * l_mix) / (4.0 * pr * sigma**2 * el_z)

    return psi

@check_units("[length]", "[pressure]", "[temperature]", "[mass]/[mass]", "[speed]", "[speed]")
def calc_ecape(
    height_msl: PintList,
    pressure: PintList,
    temperature: PintList,
    specific_humidity: PintList,
    u_wind: PintList,
    v_wind: PintList,
    cape_type: str = "most_unstable",
    undiluted_cape: pint.Quantity = None,
) -> pint.Quantity:
    """
    Calculate the entraining CAPE (ECAPE) of a parcel

    Parameters:
    ------------
        height_msl: np.ndarray[pint.Quantity]
            Atmospheric heights at the levels given by 'pressure' (MSL)
        pressure: np.ndarray[pint.Quantity]
            Total atmospheric pressure
        temperature: np.ndarray[pint.Quantity]
            Air temperature
        specific humidity: np.ndarray[pint.Quantity]
            Specific humidity
        u_wind: np.ndarray[pint.Quantity]
            X component of the wind
        v_wind np.ndarray[pint.Quantity]
            Y component of the wind
        cape_type: str
            Variation of CAPE desired. 'most_unstable' (default), 'surface_based', or 'mixed_layer'
        undiluted_cape: pint.Quantity
            User-provided undiluted CAPE value

    Returns:
    ----------
        ecape : 'pint.Quantity'
            Entraining CAPE
    """

    cape_func = {
        "most_unstable": mpcalc.most_unstable_cape_cin,
        "surface_based": mpcalc.surface_based_cape_cin,
        "mixed_layer": mpcalc.mixed_layer_cape_cin,
    }

    parcel_func = {
        "most_unstable": mpcalc.most_unstable_parcel,
        "surface_based": None,
        "mixed_layer": mpcalc.mixed_parcel,
    }

    # calculate cape
    dew_point_temperature = mpcalc.dewpoint_from_specific_humidity(pressure, temperature, specific_humidity)

    # whether the user has not / has overidden the cape calculations
    if not undiluted_cape:
        cape, _ = cape_func[cape_type](pressure, temperature, dew_point_temperature)
    else:
        cape = undiluted_cape

    # calculate the level of free convection (lfc) and equilibrium level (el) indexes
    lfc_idx, _ = calc_lfc_height(pressure, height_msl, temperature, dew_point_temperature, parcel_func[cape_type])
    el_idx, el_z = calc_el_height(pressure, height_msl, temperature, dew_point_temperature, parcel_func[cape_type])

    # calculate the buoyancy dilution potential (ncape)
    moist_static_energy_bar, moist_static_energy_star = calc_mse(pressure, height_msl, temperature, specific_humidity)
    integral_arg = calc_integral_arg(moist_static_energy_bar, moist_static_energy_star, temperature)
    ncape = calc_ncape(integral_arg, height_msl, lfc_idx, el_idx)

    # calculate the storm relative (sr) wind
    sr_wind = calc_sr_wind(pressure, u_wind, v_wind, height_msl)

    # calculate the entraining cape (ecape)
    psi = calc_psi(el_z)
    ecape_a = calc_ecape_a(sr_wind, psi, ncape, cape)

    return ecape_a

if __name__ == "__main__":
    pass

In [3]:
#UPDATED COMPOSITE PARAMETER FUNCTIONS
#These are updated sharppy functions, based on the equations for these parameters on the SPC mesoanalysis page

def scp(mucape, srh, ebwd, mucin):
    '''
    Supercell Composite Parameter (Thompson et al. 2003, 2007, 2012; Gropp and Davenport 2018)

    SCP_CIN = (muCAPE / 1000 J kg-1) * (ESRH / 50 m2 s-2) * (EBWD / 20 m s-1) * (-40 J kg-1 / muCIN)
    EBWD is divided by 20 m/s in the range of 10-20 m s-1. EBWD less than 10 m/s is set to zero, and EBWD greater than 20 m/s is set to one.
    The muCIN is set to 1.0 when muCIN is greater than -40 kg-1.

        Parameters
        ----------
        prof : profile object
            Profile object
        mucape : number, optional
            Most Unstable CAPE from the parcel class (J/kg) (optional)
        srh : number, optional
            the effective SRH from the winds.helicity function (m2/s2)
        ebwd : number, optional
            effective bulk wind difference (m/s)

        Returns
        -------
        scp : number
            supercell composite parameter   
    '''
    if ebwd > 20:
        ebwd_term = 1.0
    elif ebwd < 10:
        ebwd_term = 0.0
    else:
        ebwd_term = ebwd / 20.0
     
    if mucin > -40:
        muCIN_term = 1.0
    else:
        muCIN_term = -40.0 / mucin

    # Calculate SCP
    muCAPE_term = mucape / 1000.0
    esrh_term = srh / 50.0

    scp = muCAPE_term * esrh_term * ebwd_term * muCIN_term
    return scp

def stp_fixed(sbcape, sblcl, srh01, bwd6, sbcin):
    '''
    Significant Tornado Parameter (Thompson et al. 2012)
    STP = (sbCAPE/1500 J kg-1) * ((2000-sbLCL)/1000 m) * (SRH1/150 m2 s-2) * (6BWD/20 m s-1)* ((200+sbCIN)/150 J kg-1)

    The sbLCL term is set to 1.0 when sbLCL < 1000 m, and set to 0.0 when sbLCL > 2000 m
    the sbCIN term is set to 1.0 when sbCIN > -50 J kg-1, and set to 0.0 when sbCIN < -200
    the 6BWD term is capped at a value of 1.5 for 6BWD > 30 m/s, and set to 0.0 when 6BWD < 12.5 m/s. 
   
        Parameters
        ----------
        sbcape : number
            Surface based CAPE from the parcel class (J/kg)
        sblcl : number
            Surface based lifted condensation level (LCL) (m)
        srh01 : number
            Surface to 1 km storm relative helicity (m2/s2)
        bwd6 : number
            Bulk wind difference between 0 to 6 km (m/s)

        Returns
        -------
        stp_fixed : number
            signifcant tornado parameter (fixed-layer)
    '''
    
    # Calculate SBLCL term
    if sblcl < 1000.: # less than 1000. meters
        lcl_term = 1.0
    elif sblcl > 2000.: # greater than 2000. meters
        lcl_term = 0.0
    else:
        lcl_term = ((2000.-sblcl)/1000.)

    # Calculate 6BWD term
    if bwd6 > 30.0:  # Cap at 30 m/s
        bwd6_term = 1.5
    elif bwd6 < 12.5:  # Below 12.5 m/s
        bwd6_term = 0.0
    else:
        bwd6_term = bwd6 / 20.0

    cape_term = sbcape / 1500.
    srh_term = srh01 / 150.

    # Calculate SBCIN term
    if sbcin > -50.0:  # Less negative than -50 J/kg
        cin_term = 1.0
    elif sbcin < -200.0:  # More negative than -200 J/kg
        cin_term = 0.0
    else:
        cin_term = (200.0 + sbcin) / 150.0

    stp_fixed = cape_term * lcl_term * srh_term * bwd6_term * cin_term
   
    return stp_fixed
    
def stp_effective(mlcape, esrh, ebwd, mllcl, mlcinh, ebot_hght):
    '''
    STP effective layer (Thompson et al. 2012)
    STP = (mlCAPE/1500 J kg-1) * ((2000-mlLCL)/1000 m) * (ESRH/150 m2 s-2) * (EBWD/20 m s-1) * ((200+mlCIN)/150 J kg-1)
    The mlLCL term is set to 1.0 when mlLCL < 1000 m, and set to 0.0 when mlLCL > 2000 m
    the mlCIN term is set to 1.0 when mlCIN > -50 J kg-1, and set to 0.0 when mlCIN < -200; 
    the EBWD term is capped at a value of 1.5 for EBWD > 30 m s-1, and set to 0.0 when EBWD < 12.5 m s-1. 
    the entire index is set to 0.0 when the effective inflow base is above the ground.

        Parameters
        ----------
        mlcape : float
            Mixed-layer CAPE from the parcel class (J/kg)
        esrh : float
            effective storm relative helicity (m2/s2)
        ebwd : float
            effective bulk wind difference (m/s)
        mllcl : float
            mixed-layer lifted condensation level (m)
        mlcinh : float
            mixed-layer convective inhibition (J/kg)

        Returns
        -------
        stp_cin : number
            significant tornado parameter (unitless)

        See Also
        --------
        stp_fixed
    '''
    # If the effective inflow base is above ground, return 0.0 for STP
    if ebot_hght > 0.0:
        return 0.0

    cape_term = mlcape / 1500.
    eshr_term = esrh / 150.
    
    if ebwd < 12.5:
        ebwd_term = 0.
    elif ebwd > 30.:
        ebwd_term = 1.5
    else:
        ebwd_term  = ebwd / 20.

    if mllcl < 1000.:
        lcl_term = 1.0
    elif mllcl > 2000.:
        lcl_term = 0.0
    else:
        lcl_term = ((2000. - mllcl) / 1000.)

    if mlcinh > -50:
        cinh_term = 1.0
    elif mlcinh < -200:
        cinh_term = 0.0
    else:
        cinh_term = ((mlcinh + 200.0) / 150.0)

    stp_effective = np.maximum(cape_term * eshr_term * ebwd_term * lcl_term * cinh_term, 0.0)
    return stp_effective
 
def stp_500mSRH(mlcape, mllcl, srh_0_500m, ebwd, mlcinh, ebot_hght, etop_hght):
    """
    Calculate the STP using 0–500m SRH within the Effective Inflow Layer.
    Coffer et al. 2019: https://journals.ametsoc.org/view/journals/wefo/34/5/waf-d-19-0115_1.xml

    STP = (mlCAPE/1500 J kg-1) * ((2000-mlLCL)/1000 m) * (0-500 m SRH/75 m2 s-2) * (EBWD/20 m s-1) * ((200+mlCIN)/150 J kg-1)
    
    The 0-500 m SRH is limited to within the EIL, if it exists. 
    The mlLCL term is set to 1.0 when mlLCL < 1000 m, and set to 0.0 when mlLCL > 2000 m; 
    the mlCIN term is set to 1.0 when mlCIN > -50 J kg-1, and set to 0.0 when mlCIN < -200;
    the EBWD term is capped at a value of 1.5 for EBWD > 30 m s-1, and set to 0.0 when EBWD < 12.5 m s-1. 
    The entire index is set to 0.0 when the effective inflow base is above the ground.

    Parameters:
        mlcape (float): Mixed-layer CAPE (J/kg).
        mllcl (float): Mixed-layer LCL height (m).
        srh_0_500m (float): 0-500m storm-relative helicity (m²/s²).
        ebwd (float): Effective bulk wind difference (m/s).
        mlcinh (float): Mixed-layer CIN (J/kg).
        ebot_hght (float): Effective inflow layer base height (m).
        etop_hght (float): Effective inflow layer top height (m).

    Returns:
        float: STP value.
    """
    # If the effective inflow base is above ground, return 0.0 for STP
    if ebot_hght > 0.0:
        return 0.0

    # Limit 0-500m SRH within the effective inflow layer
    if etop_hght > 500.0:
        srh_term = srh_0_500m / 75.0
    else:
        srh_term = 0.0

    # Calculate mlLCL term
    if mllcl < 1000.0:
        lcl_term = 1.0
    elif mllcl > 2000.0:
        lcl_term = 0.0
    else:
        lcl_term = (2000.0 - mllcl) / 1000.0

    # Calculate mlCIN term
    if mlcinh > -50.0:
        cinh_term = 1.0
    elif mlcinh < -200.0:
        cinh_term = 0.0
    else:
        cinh_term = (mlcinh + 200.0) / 150.0

    # Calculate EBWD term
    if ebwd < 12.5:
        ebwd_term = 0.0
    elif ebwd > 30.0:
        ebwd_term = 1.5
    else:
        ebwd_term = ebwd / 20.0

    # Calculate other terms
    cape_term = mlcape / 1500.0

    # Final STP calculation
    stp = max(cape_term * lcl_term * srh_term * ebwd_term * cinh_term, 0.0)
    return stp

def vtp(mlcape, mllcl, esrh, ebwd, mlcinh, cape_0_3km, lapse_rate_0_3km, ebot_hght):
    """
    Calculate the Violent Tornado Parameter (VTP): https://www.spc.noaa.gov/publications/mosier/2018-JOM1.pdf.
    VTP = (mlCAPE/1500 J kg-1) * ((2000-mlLCL)/1000 m) * (ESRH/150 m2 s-2) * (EBWD/20 m s-1) * ((200+mlCIN)/150 J kg-1) * (0-3 km MLCAPE/50 J kg-1) * (0-3 km Lapse Rate/6.5 ℃ km-1)
    
    The 0-3 km lapse rate term is set to 2.0 when 0-3 km MLCAPE > 100 J kg-1.
    The mlLCL term is set to 1.0 when mlLCL < 1000 m, and set to 0 when mlLCL > 2000 m
    The mlCIN term is set to 1.0 when mlCIN > -50 J kg-1, and set to 0 when mlCIN < -200
    The EBWD term is capped at a value of 1.5 for EBWD > 30 m s-1, and set to 0 when EBWD < 12.5 m s-1.
    The entire index is set to 0 when the effective inflow base is above the ground.

    Parameters:
        mlcape (float): Mixed-layer CAPE (J/kg).
        mllcl (float): Mixed-layer LCL height (m).
        esrh (float): Effective-layer storm-relative helicity (m²/s²).
        ebwd (float): Effective bulk wind difference (m/s).
        mlcinh (float): Mixed-layer CIN (J/kg).
        cape_0_3km (float): 0–3 km mixed-layer CAPE (J/kg).
        lapse_rate_0_3km (float): 0–3 km lapse rate (°C/km).
        ebot_hght (float): Effective inflow layer base height (m).

    Returns:
        float: VTP value.
    """
    # If the effective inflow base is above ground, return 0.0 for VTP
    if ebot_hght > 0.0:
        return 0.0

    # Calculate mlLCL term
    if mllcl < 1000.0:
        lcl_term = 1.0
    elif mllcl > 2000.0:
        lcl_term = 0.0
    else:
        lcl_term = (2000.0 - mllcl) / 1000.0

    # Calculate mlCIN term
    if mlcinh > -50.0:
        cinh_term = 1.0
    elif mlcinh < -200.0:
        cinh_term = 0.0
    else:
        cinh_term = (mlcinh + 200.0) / 150.0

    # Calculate EBWD term
    if ebwd < 12.5:
        ebwd_term = 0.0
    elif ebwd > 30.0:
        ebwd_term = 1.5
    else:
        ebwd_term = ebwd / 20.0

    # Calculate 0–3 km lapse rate term
    if cape_0_3km > 100.0:
        lapse_rate_term = 2.0  # Lapse rate set to 2.0 if 0–3 km MLCAPE > 100 J/kg
    else:
        lapse_rate_term = lapse_rate_0_3km / 6.5

    # Calculate other terms
    cape_term = mlcape / 1500.0
    esrh_term = esrh / 150.0
    cape_0_3km_term = cape_0_3km / 50.0

    # Final VTP calculation
    vtp_value = max(
        cape_term
        * lcl_term
        * esrh_term
        * ebwd_term
        * cinh_term
        * cape_0_3km_term
        * lapse_rate_term,
        0.0,
    )
    return vtp_value

In [128]:
#Convergence Functions
def calculate_skewt_params(lev, hgt, T, Td, uwnd, vwnd, x, y, direction):
    """Calculate Skew-T parameters for a given grid direction."""
    if direction == "N":
        dx, dy = 1, 0
    elif direction == "S":
        dx, dy = -1, 0
    elif direction == "E":
        dx, dy = 0, 1
    elif direction == "W":
        dx, dy = 0, -1
    else:
        raise ValueError("Direction must be one of ['N', 'S', 'E', 'W']")

    p_skewt = lev[:, x + dx, y + dy] * units.hectopascal
    hgt_skewt = (hgt[:, x + dx, y + dy] - hgt[0, x + dx, y + dy]) * units.meter
    T_skewt = (T[:, x + dx, y + dy] * units.kelvin).to(units.degC)
    Td_skewt = Td[:, x + dx, y + dy].to(units.degC)
    Td_skewt[np.isnan(Td_skewt)] = 0 * units.degC
    u_skewt = (uwnd[:, x + dx, y + dy] * units.meter_per_second).to(units.knot)
    v_skewt = (vwnd[:, x + dx, y + dy] * units.meter_per_second).to(units.knot)
    #Omega = VVEL_up[:, x + dx, y + dy] * (units.pascal / units.second)
    #RH_skewt = relative_humidity_from_dewpoint(T_skewt, Td_skewt)

    #return p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt, Omega, RH_skewt
    return p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt

def calculate_parcel_profiles(p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt):
    """
    Calculate SHARPpy parcel profiles and vertical profiles for MU, ML, and SB parcels.

    Returns:
        dict: Contains SHARPpy parcel data, vertical profiles, and LFC pressures.
    """
    # Ensure consistent lengths for all inputs
    min_len = min(len(p_skewt), len(hgt_skewt), len(T_skewt), len(Td_skewt), len(u_skewt), len(v_skewt))
    p_skewt = p_skewt[:min_len]
    hgt_skewt = hgt_skewt[:min_len]
    T_skewt = T_skewt[:min_len]
    Td_skewt = Td_skewt[:min_len]
    u_skewt = u_skewt[:min_len]
    v_skewt = v_skewt[:min_len]

    # Create SHARPpy profile
    prof = profile.create_profile(
        profile="default",
        pres=p_skewt,
        hght=hgt_skewt,
        tmpc=T_skewt,
        dwpc=Td_skewt,
        u=u_skewt,
        v=v_skewt,
        missing=-9999,
    )

    # Mixed-Layer (ML) parcel
    ml_p, ml_T, ml_Td = mixed_parcel(p_skewt, T_skewt, Td_skewt)
    ml_profile = parcel_profile(p_skewt, ml_T, ml_Td)
    mlpcl = params.parcelx(prof, flag=4)  # 100 mb Mean Layer Parcel
    mllfc_p = mlpcl.lfcpres*units('hectopascal')
    if isinstance(mllfc_p, np.ma.core.MaskedConstant):
        mllfc_p = 500*units('hectopascal')

    # Most-Unstable (MU) parcel
    mu_p, mu_T, mu_Td, _ = most_unstable_parcel(p_skewt, T_skewt, Td_skewt)
    mu_profile = parcel_profile(p_skewt, mu_T, mu_Td)
    mupcl = params.parcelx(prof, flag=3)  # Most-Unstable Parcel
    mulfc_p = mupcl.lfcpres*units('hectopascal')
    if isinstance(mulfc_p,np.ma.core.MaskedConstant):
        mulfc_p = 500*units('hectopascal')

    # Surface-Based (SB) parcel
    sbpcl = params.parcelx(prof, flag=1)  # Surface-Based Parcel
    sb_profile = parcel_profile(p_skewt, T_skewt[0], Td_skewt[0])
    sblfc_p = sbpcl.lfcpres*units('hectopascal')
    if isinstance(sblfc_p, np.ma.core.MaskedConstant):
        sblfc_p = mllfc_p
    
    # Convert vertical profiles to Celsius
    sb_profile = (sb_profile - 273.15 * units.kelvin).magnitude * units.degC
    ml_profile = (ml_profile - 273.15 * units.kelvin).magnitude * units.degC
    mu_profile = (mu_profile - 273.15 * units.kelvin).magnitude * units.degC

    # Return profiles, including SHARPpy parcel data and vertical profiles
    return {
        "sb": {"parcel": sbpcl, "profile": sb_profile, "lfc_p": sblfc_p, "base":p_skewt[0]},
        "ml": {"parcel": mlpcl, "profile": ml_profile, "lfc_p": mllfc_p, "base":ml_p},
        "mu": {"parcel": mupcl, "profile": mu_profile, "lfc_p": mulfc_p, "base":mu_p},
    }

# def calculate_mean_convergence(subcloudUV_layer, distEW, distNS, parcel_h, lfc_h):
#     """
#     Calculate mean convergence for given layers, distances, and vertical levels.

#     Parameters:
#         subcloudUV_layer (dict): Dictionary with subcloud UV layers for directions ('E', 'W', 'N', 'S').
#         distEW (float): East-West distance in meters.
#         distNS (float): North-South distance in meters.
#         parcel_h (float): Parcel height (h) in meters.
#         lfc_h (float): Level of free convection (h) in meters.

#     Returns:
#         float: Mean subcloud convergence.
#     """
#     # Calculate convergence at each vertical level
#     part1 = (subcloudUV_layer['E'][1] - subcloudUV_layer['W'][1]) / (distEW * units.meter)
#     part2 = (subcloudUV_layer['N'][2] - subcloudUV_layer['S'][2]) / (distNS * units.meter)
#     convergence = part1 + part2

#     # Calculate vertical distance and mean convergence
#     vertical_distance = lfc_h - parcel_h
#     mean_convergence = np.sum(convergence) / (vertical_distance * 1000)  # Convert to kilometers

#     return mean_convergence

def calculate_distances(lat, lon, x, y):
    """Calculate distances between adjacent grid points."""
    coordN = (lat[x + 1, y], lon[x + 1, y])
    coordS = (lat[x - 1, y], lon[x - 1, y])
    coordE = (lat[x, y + 1], lon[x, y + 1])
    coordW = (lat[x, y - 1], lon[x, y - 1])

    distNS = geodesic(coordN, coordS).m
    distEW = geodesic(coordE, coordW).m
    return distNS, distEW

# def calculate_surface_convergence(subcloudUV_layer, distEW, distNS):
#     """
#     Calculate parcel level (surface) convergence for mixed-layer or most-unstable parcels.
    
#     Parameters:
#         subcloudUV_layer (dict): Sub-cloud UV layers for all directions {'N', 'S', 'E', 'W'}.
#         distEW (float): Distance between eastern and western points (meters).
#         distNS (float): Distance between northern and southern points (meters).
        
#     Returns:
#         surface_convergence (Quantity): Surface convergence value.
#     """
#     # Calculate convergence components
#     part1 = (subcloudUV_layer['E'][1][0] - subcloudUV_layer['W'][1][0]) / (distEW * units('meter'))
#     part2 = (subcloudUV_layer['N'][2][0] - subcloudUV_layer['S'][2][0]) / (distNS * units('meter'))
    
#     # Total surface convergence
#     surface_convergence = part1 + part2
#     return surface_convergence

In [129]:
# def calculate_MRD(parcel_p, lcl_p, lfc1500_p, profile):
#     # Calculate MRD1 (from starting level to LCL)
#     MRD1 = get_layer(P_sounding, T_sounding, Td_sounding, profile, RH_skewt, bottom=parcel_p, depth=parcel_p - lcl_p)
    
#     # Calculate MRD2 (from LCL to LFC + 1500 m)
#     MRD2 = get_layer(P_sounding, T_sounding, Td_sounding, profile[::-1], RH_skewt, bottom=lcl_p, depth=lcl_p - lfc1500_p)

#     # Ensure MRD1 and MRD2 contain the expected number of components
#     if len(MRD1) < 4 or len(MRD2) < 4:
#         raise ValueError("MRD1 or MRD2 does not have the expected number of layers.")

#     # Combine MRD layers
#     MRD_p = np.concatenate((MRD1[0][:-1], MRD2[0]), axis=0)  # Pressure
#     MRD_T = np.concatenate((MRD1[1][:-1], MRD2[1]), axis=0)  # Temperature
#     MRD_Td = np.concatenate((MRD1[2][:-1], MRD2[2]), axis=0)  # Dewpoint
#     MRD_pT = np.concatenate((MRD1[3][:-1], MRD2[3]), axis=0)
#     MRD_RH = np.concatenate((MRD1[4][:-1], MRD2[4]), axis=0)  # Relative Humidity
    
#     ParcelRH = 
#     Parcelr = 

#     # Calculate environmental mixing ratio
#     r_skewt_env = mixing_ratio_from_relative_humidity(MRD_p, MRD_T, MRD_RH)

#     # Step 1: Parcel mixing ratio from starting level to LCL
#     parcel_r1 = np.full(
#         np.shape(MRD1[0][:-1]),
#         mixing_ratio_from_relative_humidity(parcel_p, MRD_T[0], MRD_RH[0])
#     )

#     # Step 2: Parcel mixing ratio from LCL to LFC + 1500 m
#     parcel_r2 = mixing_ratio_from_relative_humidity(
#         MRD2[0], MRD2[2], np.full(np.shape(MRD2[0]), 1)
#     )

#     # Combine parcel mixing ratios
#     parcel_r = np.concatenate((parcel_r1, parcel_r2.magnitude), axis=0)

#     # Calculate MRD
#     MRD = np.sum(parcel_r - r_skewt_env.magnitude)
#     return MRD

In [6]:
# #Elevated Stable Layer Calculation Functions

# def compute_lapse_rates(prof, pres_array):
#     lapse_rates = []
#     for i in range(len(pres_array) - 1):
#         lapse = sharppy.sharptab.params.lapse_rate(prof, lower=pres_array[i], upper=pres_array[i + 1])
#         lapse_rates.append(lapse)
#     return np.array(lapse_rates)

# # Function to compute heights for a given pressure array
# def compute_heights(prof, pres_array):
#     heights = [sharppy.sharptab.interp.hght(prof, p) for p in pres_array]
#     return np.array(heights)

# # Function to calculate ESL depths
# def compute_esl_depth(lapse_array, hght_array, threshold):
#     esl_indices = np.where(lapse_array < threshold)[0]
#     if len(esl_indices) > 0:
#         esl_depth = hght_array[esl_indices[-1]] - hght_array[esl_indices[0]]
#     else:
#         esl_depth = 0
#     return esl_depth

# pres_array = np.array([900., 890., 880., 870., 860., 850., 840., 830., 820., 810., 800.,
#                        790., 780., 770., 760., 750., 740., 730., 720., 710., 700., 690.,
#                        680., 670., 660., 650., 640., 630., 620., 610., 600.])

In [9]:
##########################################################
# Pre Load HRRR Files to speed up parameter calculations #
##########################################################
def download_and_open_hrrr(year, month, day, hours, s3_path='s3://noaa-hrrr-bdp-pds', level_filter='hybrid'):
    """
    Download and open HRRR GRIB2 files for specified hours.
    
    Parameters:
        year (int): Year of the data
        month (int): Month of the data
        day (int): Day of the data
        hours (list): List of hours to download
        s3_path (str): Base S3 path to HRRR files
        level_filter (str): Filter for the level type (e.g., 'hybrid')
    
    Returns:
        dict: Dictionary of Xarray datasets
    """
    fs = s3fs.S3FileSystem(anon=True)
    datasets = {}
    
    for idx, hour in enumerate(hours):
        # Format the year, month, day, and hour
        year_str = f"{year}"
        month_str = f"{month:02}"
        day_str = f"{day:02}"
        hour_str = f"{hour:02}"
        
        # Construct the file path and local filename
        time_in = f"{year_str}{month_str}{day_str}"
        file_path = f"{s3_path}/hrrr.{time_in}/conus/hrrr.t{hour_str}z.wrfnatf00.grib2"
        local_fname = f"HRRR_{year_str}{month_str}{day_str}_{hour_str}UTC.grib2"
        
        # Download the file
        if not os.path.exists(local_fname):  # Avoid re-downloading
            fs.get(file_path, local_fname)
        
        # Open the file with Xarray
        datasets[f"c{idx}"] = xr.open_dataset(local_fname, filter_by_keys={'typeOfLevel': level_filter})
    
    return datasets

# Parameters
year = 2019
month = 5

# Download data for 22nd May
hours_22 = [18, 19, 20, 21, 22, 23]
d1 = download_and_open_hrrr(year, month, 22, hours_22)

# Download data for 23rd May
hours_23 = [0, 1, 2, 3, 4, 5, 6, 7, 8]
d2 = download_and_open_hrrr(year, month, 23, hours_23)

In [7]:
# Define all parameter names as a flat list
parameters = [
    'StormID', 'Profile_Number', 'Time', 'Watch', 'Grouping', 'Radar', 'Lat', 'Lon', 'Type', 'Discrete',
    'Supercell', 'LeftSplit', 'Severe30', 'ConfirmedTor30', 'Profile', 'MaxProfiles', 'Daylight',
    'SupercellAny', 'SevereAny', 'TorAny', 'Start', 'End', 'Life (in mins)', 'Demise', 'Ini_Lat', 'Ini_Lon',  
    'cenlat', 'cenlon', 'ModelLat', 'ModelLon', 'Prof_QC_issue_(bool)',
    
    'sbLCL','mlLCL','muLCL','sbLFC','mlLFC','muLFC','sb_LFC_LCL_diff','ml_LFC_LCL_diff','mu_LFC_LCL_diff',
    'pbl_top','EIL_depth','EIL_PBL_hgtdiff','sbMaxOmegaH','mlMaxOmegaH','muMaxOmegaH',
    'sbHLFC','mlHLFC', 'muHLFC', 'deltaZstar',
    
    'sbMaxOmega','mlMaxOmega','muMaxOmega','sbSCconv','mlSCconv','muSCconv','sbPLconv','mlPLconv','muPLconv',
    
    'sfc1kmLapse','sfc2kmLapse','sfc3kmLapse','3_6kmLapse','5_700mbLapse','sbACBLlapse','mlACBLlapse','muACBLlapse',
    'sb_lapse_LFC_LCL','ml_lapse_LFC_LCL','mu_lapse_LFC_LCL','esl_25_depth','esl_40_depth',
    
    'sbMRD','mlMRD','muMRD',
    'pwat',
    'ThetaEindex',
    'maxTE3kmhgt','maxQ3kmhgt',
    'meanRH100mb','meanQ100mb','meanTe100mb',
    'meanRH1km','meanQ1km','meanTe1km',
    'meanRH150mb','meanQ150mb','meanTe150mb',
    'meanRH150_300mb','meanQ150_300mb','meanTe150_300mb',
    'meanRHpbl','meanQpbl','meanTepbl',
    'meanRH1_3km','meanQ1_3km','meanTe1_3km',
    'meanRH_EIL','meanQ_EIL','meanTe_EIL',
    'sb_meanRH_LFC_LCL','ml_meanRH_LFC_LCL','mu_meanRH_LFC_LCL',
    'sb_meanQ_LFC_LCL','ml_meanQ_LFC_LCL','mu_meanQ_LFC_LCL',
    'sb_meanTe_LFC_LCL','ml_meanTe_LFC_LCL','mu_meanTe_LFC_LCL',
    'meanRHsb_ACBL','meanRHml_ACBL','meanRHmu_ACBL',
    'meanQsb_ACBL','meanQml_ACBL','meanQmu_ACBL',
    'meanTesb_ACBL','meanTeml_ACBL','meanTemu_ACBL',
    'meanRHsb_LCL2k','meanRHml_LCL2k','meanRHmu_LCL2k',
    'meanQsb_LCL2k','meanQml_LCL2k','meanQmu_LCL2k',
    'meanTesb_LCL2k','meanTeml_LCL2k','meanTemu_LCL2k',
    '500m_3km_RH', '500m_4km_RH', '500m_5km_RH', '500m_6km_RH',
    '1000m_3km_RH', '1000m_4km_RH', '1000m_5km_RH', '1000m_6km_RH',
    '1500m_3km_RH', '1500m_4km_RH', '1500m_5km_RH', '1500m_6km_RH',
    '2000m_3km_RH', '2000m_4km_RH', '2000m_5km_RH', '2000m_6km_RH',
    '2500m_3km_RH', '2500m_4km_RH', '2500m_5km_RH', '2500m_6km_RH',
    '3000m_4km_RH', '3000m_5km_RH', '3000m_6km_RH',
    
    'sbCAPE','mlCAPE','muCAPE',
    'sb_LCLCAPE','ml_LCLCAPE','mu_LCLCAPE',
    'sb3CAPE','ml3CAPE','mu3CAPE',
    'sb6CAPE','ml6CAPE','mu6CAPE','sbFrzCAPE','mlFrzCAPE','muFrzCAPE',
    'dCAPE',
    'sb_LImax','ml_LImax','mu_LImax',
    'eCAPE',
    'sbCAPE_500m_3km','sbCAPE_500m_4km','sbCAPE_500m_5km','sbCAPE_500m_6km',
    'mlCAPE_500m_3km','mlCAPE_500m_4km','mlCAPE_500m_5km','mlCAPE_500m_6km',
    'muCAPE_500m_3km','muCAPE_500m_4km','muCAPE_500m_5km','muCAPE_500m_6km',
    'sbCAPE_1_3km','sbCAPE_1_4km','sbCAPE_1_5km','sbCAPE_1_6km',
    'mlCAPE_1_3km','mlCAPE_1_4km','mlCAPE_1_5km','mlCAPE_1_6km',
    'muCAPE_1_3km','muCAPE_1_4km','muCAPE_1_5km','muCAPE_1_6km',
    'sbCAPE_1500m_3km','sbCAPE_1500m_4km','sbCAPE_1500m_5km','sbCAPE_1500m_6km',
    'mlCAPE_1500m_3km','mlCAPE_1500m_4km','mlCAPE_1500m_5km','mlCAPE_1500m_6km',
    'muCAPE_1500m_3km','muCAPE_1500m_4km','muCAPE_1500m_5km','muCAPE_1500m_6km',
    'sbCAPE_2_3km','sbCAPE_2_4km','sbCAPE_2_5km','sbCAPE_2_6km',
    'mlCAPE_2_3km','mlCAPE_2_4km','mlCAPE_2_5km','mlCAPE_2_6km',
    'muCAPE_2_3km','muCAPE_2_4km','muCAPE_2_5km','muCAPE_2_6km',
    
    'sbCIN','mlCIN','muCIN',
    'ConvectiveT','700mbTemp',
    'sbCIN_500m_3km','sbCIN_500m_4km','sbCIN_500m_5km','sbCIN_500m_6km',
    'mlCIN_500m_3km','mlCIN_500m_4km','mlCIN_500m_5km','mlCIN_500m_6km',
    'muCIN_500m_3km','muCIN_500m_4km','muCIN_500m_5km','muCIN_500m_6km',
    'sbCIN_1_3km','sbCIN_1_4km','sbCIN_1_5km','sbCIN_1_6km',
    'mlCIN_1_3km','mlCIN_1_4km','mlCIN_1_5km','mlCIN_1_6km',
    'muCIN_1_3km','muCIN_1_4km','muCIN_1_5km','muCIN_1_6km',
    'sbCIN_1500m_3km','sbCIN_1500m_4km','sbCIN_1500m_5km','sbCIN_1500m_6km',
    'mlCIN_1500m_3km','mlCIN_1500m_4km','mlCIN_1500m_5km','mlCIN_1500m_6km',
    'muCIN_1500m_3km','muCIN_1500m_4km','muCIN_1500m_5km','muCIN_1500m_6km',
    'sbCIN_2_3km','sbCIN_2_4km','sbCIN_2_5km','sbCIN_2_6km',
    'mlCIN_2_3km','mlCIN_2_4km','mlCIN_2_5km','mlCIN_2_6km',
    'muCIN_2_3km','muCIN_2_4km','muCIN_2_5km','muCIN_2_6km',
    
    'bunkersRMU','bunkersRMV','bunkersRM','bunkersRMangle',
    
    'sfc500mSRH','sfc1kmSRH','sfc3kmSRH','ESRH',
    
    '0_500_m_BWD','0_1_km_BWD','0_2_km_BWD','0_3_km_BWD','0_5_km_BWD', '0_6_km_BWD',
    '0_8_km_BWD','EIL_BWD','sb_Effective_BWD','ml_Effective_BWD','mu_Effective_BWD',
    'sb_LCL_EL_BWD','ml_LCL_EL_BWD','mu_LCL_EL_BWD',
    
    '0_500mSRflow','0_1kmSRflow','0_2kmSRflow','0_3kmSRflow','0_6kmSRflow','0_8kmSRflow',
    '4_5kmSRflow','4_6kmSRflow','sb_LCL_EL_SRflow','ml_LCL_EL_SRflow','mu_LCL_EL_SRflow','EIL_SRflow',
    
    'sbSCshear','mlSCshear','muSCshear','sbACBLshear','mlACBLshear','muACBLshear',
    'critAngle','sb_brnSHR','ml_brnSHR','mu_brnSHR',
    'MeanShearValue_500m','MeanSWvalue_500m','MeanSWpercvalue_500m',
    'MeanShearValue_1km','MeanSWvalue_1km','MeanSWpercvalue_1km',
    'MeanShearValue_3km','MeanSWvalue_3km','MeanSWpercvalue_3km',
    '1_3km_Shear','1_3km_SW','1_3km_SW_perc',
    
    'SCP','STP_FL','STP_EL','STP_500','VTP','sbBRN','mlBRN','muBRN','DCP',
    'WDP','SHP','SigSev','sb01ehi','sb03ehi','ml01ehi','ml03ehi','mu01ehi','mu03ehi'
]

# Create the DataFrame
df1 = pd.DataFrame(columns=parameters)

In [130]:
dfread= pd.read_excel('D:/THESIS/__UNL MS Thesis (MAIN)__/3 - Obtain Vertical Profiles/5-22_NearInflowPoints.xlsx')

year=2019
month=5

for j in range(0, len(dfread)):
    hour = dfread['Time'].astype("int64")
    dfread['day'] = dfread['Time'].apply(lambda x: '22' if x > 15 else '23').astype(int)
    day = dfread['day'].astype("int64")
    cenlat = dfread['Lat'].astype("int64")
    cenlon = dfread['Lon'].astype("int64")
    StormID = dfread['StormID'].astype('str')

data = dfread.to_numpy()
for i in range(0, len(data)):
    StormID = str(data[i][0])
    hour = int(data[i][1])
    cenlat = (data[i][5])
    cenlon = (data[i][6])
    day = int(data[i][-1])

    if i < 10:
        i1 = '%02d'%i
    else:
        i1 = i
        
    prof_num = i1
    
    if hour==18:
        c=d1['c0']
    elif hour==19:
        c=d1['c1']
    elif hour==20:
        c=d1['c2']
    elif hour==21:
        c=d1['c3']
    elif hour==22:
        c=d1['c4']
    elif hour==23:
        c=d1['c5']
    elif hour==0:
        c=d2['c0'] 
    elif hour==1:
        c=d2['c1'] 
    elif hour==2:
        c=d2['c2'] 
    elif hour==3:
        c=d2['c3']     
    elif hour==4:
        c=d2['c4']       
    elif hour==5:
        c=d2['c5']
    elif hour==6:
        c=d2['c6']
    elif hour==7:
        c=d2['c7']
    elif hour==8:
        c=d2['c8']
    else:
        print("WRONG HOUR")

    time_start = datetime(int(year), int(month), int(day), int(hour), 0) # Our specified time
    hour = time_start.hour
    if hour < 10:
        hour = '0'+str(hour)
    day = time_start.day
    if day < 10:
        day = '0'+str(day)
    month = time_start.month
    if month < 10:
        month = '0'+str(month)
        
    timeIN=str(str(year)+str(month)+str(day))
       
    #################################################################################################################
    lev=np.asarray(c.variables['hybrid'][:]) # 0 = lowest level, 49 = highest level
    lat=np.asarray(c.variables['latitude'])
    lon=((np.asarray(c.variables['longitude'])*-1)+360)*-1

    T=np.asarray(c.variables['t'][:]) #K #temperature
    q=np.asarray(c.variables['q'][:]) # kg kg**-1 #specific humidity
    uwnd=np.asarray(c.variables['u'][:]) #m/s #u wind
    vwnd=np.asarray(c.variables['v'][:]) #m/s #v wind
    hgt=np.asarray(c.variables['gh'][:]) #geopotential meters #geopotential height
    VVEL_up=(np.asarray(c.variables['w'][:])) #Pa s**-1 #vertical velocity  
    lev=(np.asarray(c.variables['pres'][:])/100.) #hPa #pressure
    Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)

    dlon = np.abs(lon - cenlon)
    dlat = np.abs(lat - cenlat)
    cordmax = np.maximum(dlon,dlat)
    x,y=np.where(cordmax == np.min(cordmax))
    x=x[0]
    y=y[0]

    print(cenlat,cenlon)
    print(lat[x][y],lon[x][y], "\n")

    P_sounding = lev[:,x,y]* units.hectopascal
    T_sounding = (T[:,x,y]* units.kelvin).to(units.degC)
    Td_sounding = Td[:,x,y].to(units.degC)
    Td_sounding[np.isnan(Td_sounding)] = 0*units('degC')
    hgt_sounding = (hgt[:,x,y] - hgt[0,x,y])* units.meter
    msl_sounding = (hgt[:,x,y])* units.meter
    u_sounding = (uwnd[:,x,y]* units.meter_per_second).to(units.knot)
    v_sounding = (vwnd[:,x,y]* units.meter_per_second).to(units.knot)
    Omega=VVEL_up[:,x,y]*(units.pascal/units.second)
    RH_skewt = relative_humidity_from_dewpoint(T_sounding, Td_sounding)
    Q_skewt = mixing_ratio_from_relative_humidity(P_sounding, T_sounding, RH_skewt)

    prof = profile.create_profile(profile='default', pres=P_sounding, hght=hgt_sounding, tmpc=T_sounding, dwpc=Td_sounding, u=u_sounding, v=v_sounding, missing=-9999)
    mupcl = params.parcelx(prof, flag=3) # Most-Unstable Parcel
    mlpcl = params.parcelx(prof, flag=4)
    sbpcl = params.parcelx(prof, flag=1)

    #EIL Stuff
    EILb, EILt = sharppy.sharptab.params.effective_inflow_layer(prof)
    ebot_hght = interp.to_agl(prof, interp.hght(prof, EILb))
    etop_hght = interp.to_agl(prof, interp.hght(prof, EILt))

    #METPY PARCEL INFORMATION
    #Interpolate new vertical profiles with the mlpcl and mupcl pressures, temps, and dewpoints as the starting values for each
    ml_p, ml_T, ml_Td = mixed_parcel(P_sounding, T_sounding, Td_sounding) #Pres, Temp, and Td of a 100 hPa depth mixed parcels
    ml_profile = parcel_profile(P_sounding, ml_T, ml_Td).to(units.degC)
    ml_parcel_h = pressure_to_height_std(ml_p)
    ml_parcel_h = ml_parcel_h.to(units.meter)

    mu_p, mu_T, mu_Td, mu_index = most_unstable_parcel(P_sounding, T_sounding, Td_sounding)
    mu_profile = parcel_profile(P_sounding, mu_T, mu_Td).to(units.degC)
    mu_parcel_h = pressure_to_height_std(mu_p)
    mu_parcel_h = mu_parcel_h.to(units.meter)
    
    sb_profile = parcel_profile(P_sounding, T_sounding[0], Td_sounding[0])
    
    sfc = prof.pres[prof.sfc]
    p500m = interp.pres(prof, interp.to_msl(prof, 500.))
    p1km = interp.pres(prof, interp.to_msl(prof, 1000.))
    p1500m = interp.pres(prof, interp.to_msl(prof, 1500.))
    p2km = interp.pres(prof, interp.to_msl(prof, 2000.))
    p2500m = interp.pres(prof, interp.to_msl(prof, 2500.))
    p3km = interp.pres(prof, interp.to_msl(prof, 3000.))
    p4km = interp.pres(prof, interp.to_msl(prof, 4000.))
    p5km = interp.pres(prof, interp.to_msl(prof, 5000.))
    p6km = interp.pres(prof, interp.to_msl(prof, 6000.))
    p8km = interp.pres(prof, interp.to_msl(prof, 8000.))
    p9km = interp.pres(prof, interp.to_msl(prof, 9000.)) 
    #################################################################################################################
    #LIFTING HEIGHTS and VERTICAL MOTION
    #################################################################################################################
    mllcl_p = mlpcl.lclpres*units('hectopascal')
    if isinstance(mllcl_p, np.ma.core.MaskedConstant):
        mllcl_p = 600*units('hectopascal')
    mllcl_h = mlpcl.lclhght*units('meter')
    if isinstance(mllcl_h, np.ma.core.MaskedConstant):
        mllcl_h = 4000*units('meter')
    mulcl_p = mupcl.lclpres*units('hectopascal')
    if isinstance(mulcl_p, np.ma.core.MaskedConstant):
        mulcl_p = 600*units('hectopascal')
    mulcl_h = mupcl.lclhght*units('meter')
    if isinstance(mulcl_h, np.ma.core.MaskedConstant):
        mulcl_h = 4000*units('meter')
    sblcl_p = sbpcl.lclpres*units('hectopascal')
    if isinstance(sblcl_p, np.ma.core.MaskedConstant):
        sblcl_p = mllcl_p
    sblcl_h = sbpcl.lclhght*units('meter')
    if isinstance(sblcl_h, np.ma.core.MaskedConstant):
        sblcl_h = mllcl_h

    mllfc_p = mlpcl.lfcpres *units('hectopascal')
    if isinstance(mllfc_p, np.ma.core.MaskedConstant):
        mllfc_p = 500*units('hectopascal')
    mllfc_h = mlpcl.lfchght *units('meter')
    if isinstance(mllfc_h, np.ma.core.MaskedConstant):
        mllfc_h = 5000*units('meter')
    mulfc_p = mupcl.lfcpres *units('hectopascal')
    if isinstance(mulfc_p, np.ma.core.MaskedConstant):
        mulfc_p = 500*units('hectopascal')
    mulfc_h = mupcl.lfchght *units('meter')
    if isinstance(mulfc_h, np.ma.core.MaskedConstant):
        mulfc_h = 5000*units('meter')
    sblfc_p = sbpcl.lfcpres *units('hectopascal')
    if isinstance(sblfc_p, np.ma.core.MaskedConstant):
        sblfc_p = mllfc_p
    sblfc_h = sbpcl.lfchght *units('meter')
    if isinstance(sblfc_h, np.ma.core.MaskedConstant):
        sblfc_h = mllfc_h
        
    #LCL + 2000m calculationsa
    sblcl2000 = sblcl_h + 2000*units('meter')
    mllcl2000 = mllcl_h + 2000*units('meter')
    mulcl2000 = mulcl_h + 2000*units('meter')
    sblcl2000_p = height_to_pressure_std(sblcl2000)
    mllcl2000_p = height_to_pressure_std(mllcl2000)
    mulcl2000_p = height_to_pressure_std(mulcl2000)
        
    #ACBL = 1.5 km deep layer, starting @ LFC
    sblfc_h1500 = sblfc_h + 1500*units('meter')
    mllfc_h1500 = mllfc_h + 1500*units('meter')
    mulfc_h1500 = mulfc_h + 1500*units('meter')
    sblfc1500_p = height_to_pressure_std(sblfc_h1500)
    mllfc1500_p = height_to_pressure_std(mllfc_h1500)
    mulfc1500_p = height_to_pressure_std(mulfc_h1500)
        
    sb_LFC_LCL_diff = sblfc_h - sblcl_h
    ml_LFC_LCL_diff = mllfc_h - mllcl_h
    mu_LFC_LCL_diff = mulfc_h - mulcl_h
    
    pbl_top = sharppy.sharptab.params.pbl_top(prof) #in mb
    EIL_depth = etop_hght-ebot_hght
    #EIL base vs. PBL top
    pbl_topHgt = pressure_to_height_std(pbl_top*units('hectopascal')).to(units.meter)
    EIL_PBL_hgtdiff = pbl_topHgt - ebot_hght*units('meter') #top of PBL - bottom of EIL. Negative value = EIL is fully above PBL

    #magnitude of the maximum omega value that is no more than 100 mb above the LFC
    SBmaxOmegalev = P_sounding[np.array(P_sounding>(sblfc_p- 100*units('hectopascal')))]
    SBHvalue = (np.size(SBmaxOmegalev))
    sbMaxOmega = (np.min(Omega[0:SBHvalue]))
    #height of the maximum omega value that is no more than 100 mb above the LFC
    SBresult = np.where(Omega == sbMaxOmega)
    SBMaxOmegaPressure = P_sounding[SBresult[0]]
    SBMaxOmegaHeight = pressure_to_height_std(P_sounding[SBresult[0]]).to(units.meter)
    sbMaxOmegaH = SBMaxOmegaHeight[0]
    sbHLFC = (SBMaxOmegaHeight/sblfc_h).magnitude
    
    MLmaxOmegalev = P_sounding[np.array(P_sounding>(mllfc_p- 100*units('hectopascal')))]
    MLHvalue = (np.size(MLmaxOmegalev))
    mlMaxOmega = (np.min(Omega[0:MLHvalue]))
    #height of the maximum omega value that is no more than 100 mb above the LFC
    MLresult = np.where(Omega == mlMaxOmega)
    MLMaxOmegaPressure = P_sounding[MLresult[0]]
    MLMaxOmegaHeight = pressure_to_height_std(P_sounding[MLresult[0]]).to(units.meter)
    mlMaxOmegaH = MLMaxOmegaHeight[0]
    mlHLFC = (MLMaxOmegaHeight/mllfc_h).magnitude

    MUmaxOmegalev = P_sounding[np.array(P_sounding>(mulfc_p- 100*units('hectopascal')))]
    MUHvalue = (np.size(MUmaxOmegalev))
    muMaxOmega = (np.min(Omega[0:MUHvalue]))
    #height of the maximum omega value that is no more than 100 mb above the LFC
    MUresult = np.where(Omega == muMaxOmega)
    MUMaxOmegaPressure = P_sounding[MUresult[0]]
    MUMaxOmegaHeight = pressure_to_height_std(P_sounding[MUresult[0]]).to(units.meter)
    muMaxOmegaH = MUMaxOmegaHeight[0]
    muHLFC = (MUMaxOmegaHeight/mulfc_h).magnitude
    #########################################################
    # LFC Height - initial parcel height
    mu_parcel_h = interp.to_agl(prof, interp.hght(prof,mupcl.pres))
    deltaZstar = mulfc_h.magnitude - mu_parcel_h
    ##############################   CONVERGENCE   ##############################
    # Assuming data arrays `lev`, `hgt`, `T`, `Td`, `uwnd`, `vwnd`, `VVEL_up`, `lat`, `lon`, and (x, y) coordinates
    directions = ["N", "S", "E", "W"]

    # Calculate skewt parameters for all directions
    parameters = {}
    for direction in directions:
        parameters[direction] = calculate_skewt_params(lev, hgt, T, Td, uwnd, vwnd, x, y, direction)

    # Calculate parcel profiles for all directions
    parcel_profiles = {"ml": {}, "mu": {}, "sb": {}}
    lfc_pressures = {"ml": {}, "mu": {}, "sb": {}}  # Store LFC pressures for all parcels
    bases = {"ml": {}, "mu": {}, "sb": {}}

    for direction in directions:
        # Extract Skew-T parameters
        p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt, *_ = parameters[direction]

        # Calculate profiles
        profiles = calculate_parcel_profiles(p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt)

        for parcel_type in ["ml", "mu", "sb"]:
            parcel_profiles[parcel_type][direction] = profiles[parcel_type]["profile"]
            lfc_pressures[parcel_type][direction] = profiles[parcel_type]["lfc_p"]
            bases[parcel_type][direction] = profiles[parcel_type]["base"]

    # Calculate distances between adjacent grid points
    distNS, distEW = calculate_distances(lat, lon, x, y)

    # Calculate sub-cloud UV layers for all parcel types and directions
    sb_subcloudUV_layerN = get_layer(parameters["N"][0], parameters["N"][4], parameters["N"][5], 
                                     parcel_profiles['sb']['N'], bottom=bases['sb']['N'], 
                                     depth=bases['sb']['N'] - lfc_pressures['sb']['N'])
    sb_subcloudUV_layerS = get_layer(parameters["S"][0], parameters["S"][4], parameters["S"][5], 
                                     parcel_profiles['sb']['S'], bottom=bases['sb']['S'], 
                                     depth=bases['sb']['S'] - lfc_pressures['sb']['S'])
    sb_subcloudUV_layerE = get_layer(parameters["E"][0], parameters["E"][4], parameters["E"][5], 
                                     parcel_profiles['sb']['E'], bottom=bases['sb']['E'], 
                                     depth=bases['sb']['E'] - lfc_pressures['sb']['E'])
    sb_subcloudUV_layerW = get_layer(parameters["W"][0], parameters["W"][4], parameters["W"][5], 
                                     parcel_profiles['sb']['W'], bottom=bases['sb']['W'], 
                                     depth=bases['sb']['W'] - lfc_pressures['sb']['W'])
    ml_subcloudUV_layerN = get_layer(parameters["N"][0], parameters["N"][4], parameters["N"][5], 
                                     parcel_profiles['ml']['N'], bottom=bases['ml']['N'], 
                                     depth=bases['ml']['N'] - lfc_pressures['ml']['N'])
    ml_subcloudUV_layerS = get_layer(parameters["S"][0], parameters["S"][4], parameters["S"][5], 
                                     parcel_profiles['ml']['S'], bottom=bases['ml']['S'], 
                                     depth=bases['ml']['S'] - lfc_pressures['ml']['S'])
    ml_subcloudUV_layerE = get_layer(parameters["E"][0], parameters["E"][4], parameters["E"][5], 
                                     parcel_profiles['ml']['E'], bottom=bases['ml']['E'], 
                                     depth=bases['ml']['E'] - lfc_pressures['ml']['E'])
    ml_subcloudUV_layerW = get_layer(parameters["W"][0], parameters["W"][4], parameters["W"][5], 
                                     parcel_profiles['ml']['W'], bottom=bases['ml']['W'], 
                                     depth=bases['ml']['W'] - lfc_pressures['ml']['W'])
    mu_subcloudUV_layerN = get_layer(parameters["N"][0], parameters["N"][4], parameters["N"][5], 
                                     parcel_profiles['mu']['N'], bottom=bases['mu']['N'], 
                                     depth=bases['mu']['N'] - lfc_pressures['mu']['N'])
    mu_subcloudUV_layerS = get_layer(parameters["S"][0], parameters["S"][4], parameters["S"][5], 
                                     parcel_profiles['mu']['S'], bottom=bases['mu']['S'], 
                                     depth=bases['mu']['S'] - lfc_pressures['mu']['S'])
    mu_subcloudUV_layerE = get_layer(parameters["E"][0], parameters["E"][4], parameters["E"][5], 
                                     parcel_profiles['mu']['E'], bottom=bases['mu']['E'], 
                                     depth=bases['mu']['E'] - lfc_pressures['mu']['E'])
    mu_subcloudUV_layerW = get_layer(parameters["W"][0], parameters["W"][4], parameters["W"][5], 
                                     parcel_profiles['mu']['W'], bottom=bases['mu']['W'], 
                                     depth=bases['mu']['W'] - lfc_pressures['mu']['W'])
    
    list0 = [len(sb_subcloudUV_layerE[1]),len(sb_subcloudUV_layerW[1]),len(sb_subcloudUV_layerN[2]), len(sb_subcloudUV_layerS[2])]
    list1 = [len(ml_subcloudUV_layerE[1]),len(ml_subcloudUV_layerW[1]),len(ml_subcloudUV_layerN[2]), len(ml_subcloudUV_layerS[2])]
    list2 = [len(mu_subcloudUV_layerE[1]),len(mu_subcloudUV_layerW[1]),len(mu_subcloudUV_layerN[2]), len(mu_subcloudUV_layerS[2])]
    
    sb_part1 = (sb_subcloudUV_layerE[1][:min(list0)] - sb_subcloudUV_layerW[1][:min(list0)])/(distEW*units('meter'))
    sb_part2 = (sb_subcloudUV_layerN[2][:min(list0)] - sb_subcloudUV_layerS[2][:min(list0)])/(distNS*units('meter'))
    ml_part1 = (ml_subcloudUV_layerE[1][:min(list1)] - ml_subcloudUV_layerW[1][:min(list1)])/(distEW*units('meter'))
    ml_part2 = (ml_subcloudUV_layerN[2][:min(list1)] - ml_subcloudUV_layerS[2][:min(list1)])/(distNS*units('meter'))
    mu_part1 = (mu_subcloudUV_layerE[1][:min(list2)] - mu_subcloudUV_layerW[1][:min(list2)])/(distEW*units('meter'))
    mu_part2 = (mu_subcloudUV_layerN[2][:min(list2)] - mu_subcloudUV_layerS[2][:min(list2)])/(distNS*units('meter'))

    sb_div = sb_part1 + sb_part2
    sb_div_dis = sblfc_h
    sb_SCconv = np.sum(sb_div)/((sblfc_h.magnitude)*1000)
    
    ml_div = ml_part1 + ml_part2
    ml_div_dis = mllfc_h - ml_parcel_h
    ml_SCconv = np.sum(ml_div)/((ml_div_dis.magnitude)*1000)
    
    mu_div = mu_part1 + mu_part2
    mu_div_dis = mulfc_h - mu_parcel_h*units('meter')
    mu_SCconv = np.sum(mu_div)/((mu_div_dis.magnitude)*1000)
    
    #CALCULATE Initial parcel Level (aka Surface) CONVERGENCE
    sb_PL_part1 = (sb_subcloudUV_layerE[1][0] - sb_subcloudUV_layerW[1][0])/(distEW*units('meter'))
    sb_PL_part2 = (sb_subcloudUV_layerN[2][0] - sb_subcloudUV_layerS[2][0])/(distNS*units('meter'))
    sb_PL_conv = sb_PL_part1 + sb_PL_part2
    
    ml_PL_part1 = (ml_subcloudUV_layerE[1][0] - ml_subcloudUV_layerW[1][0])/(distEW*units('meter'))
    ml_PL_part2 = (ml_subcloudUV_layerN[2][0] - ml_subcloudUV_layerS[2][0])/(distNS*units('meter'))
    ml_PL_conv = ml_PL_part1 + ml_PL_part2

    mu_PL_part1 = (mu_subcloudUV_layerE[1][0] - mu_subcloudUV_layerW[1][0])/(distEW*units('meter'))
    mu_PL_part2 = (mu_subcloudUV_layerN[2][0] - mu_subcloudUV_layerS[2][0])/(distNS*units('meter'))
    mu_PL_conv = mu_PL_part1 + mu_PL_part2
    #################################################################################################################
    #LAPSE RATES
    #################################################################################################################
    #ACBL = 1.5 km deep layer, starting @ LFC
    #1) get LFC +1500m height (use LFC height calculation from earlier)
    lapse0_1km = sharppy.sharptab.params.lapse_rate(prof, lower = sfc, upper =p1km)
    lapse0_2km = sharppy.sharptab.params.lapse_rate(prof, lower = sfc, upper =p2km)
    
    lapse0_3km = sharppy.sharptab.params.lapse_rate(prof, lower = sfc, upper =p3km)
    lapse3_6km = sharppy.sharptab.params.lapse_rate(prof, lower = p3km, upper =p6km)  
    lapse5_700mb = sharppy.sharptab.params.lapse_rate(prof, lower = 700., upper =500.)
    
    SBLFCTemp = (metinterp(sblfc_h, hgt_sounding, T_sounding)[0].magnitude)*units('degC')
    SBLFCTemp1500 = (metinterp(sblfc_h1500, hgt_sounding, T_sounding)[0].magnitude)*units('degC')
    MLLFCTemp = (metinterp(mllfc_h, hgt_sounding, T_sounding)[0].magnitude)*units('degC')
    MLLFCTemp1500 = (metinterp(mllfc_h1500, hgt_sounding, T_sounding)[0].magnitude)*units('degC')
    MULFCTemp = (metinterp(mulfc_h, hgt_sounding, T_sounding)[0].magnitude)*units('degC')
    MULFCTemp1500 = (metinterp(mulfc_h1500, hgt_sounding, T_sounding)[0].magnitude)*units('degC')
    
    #change in temp/change in height = lapse rate
    sbACBLlapse = ((SBLFCTemp1500 - SBLFCTemp)/(-1.5))
    mlACBLlapse = ((MLLFCTemp1500 - MLLFCTemp)/(-1.5))
    muACBLlapse = ((MULFCTemp1500 - MULFCTemp)/(-1.5))

    sb_lapse_LFC_LCL = sharppy.sharptab.params.lapse_rate(prof, lower = sblcl_p.magnitude, upper=sblfc_p.magnitude)
    ml_lapse_LFC_LCL = sharppy.sharptab.params.lapse_rate(prof, lower = mllcl_p.magnitude, upper=mllfc_p.magnitude)
    mu_lapse_LFC_LCL = sharppy.sharptab.params.lapse_rate(prof, lower = mulcl_p.magnitude, upper=mulfc_p.magnitude)

    if mulcl_p.magnitude == mulfc_p.magnitude:
        mu_lapse_LFC_LCL = 0
    if mllcl_p.magnitude == mllfc_p.magnitude:
        ml_lapse_LFC_LCL = 0
    if sblcl_p.magnitude == sblfc_p.magnitude:
        sb_lapse_LFC_LCL = 0

    lapse_array = compute_lapse_rates(prof, pres_array)
    hght_array = compute_heights(prof, pres_array)

    # Compute ESL depths
    presArray=np.array([890.,880.,870.,860.,850.,840.,830.,820.,810.,800.,790.,780.,770.,760.,750.,740.,730.,720.,710.,700.,
                   690.,680.,670.,660.,650.,640.,630.,620.,610.,600.])

    lapse1 = sharppy.sharptab.params.lapse_rate(prof, lower = 900., upper =890.)
    lapse2 = sharppy.sharptab.params.lapse_rate(prof, lower = 890., upper =880.)
    lapse3 = sharppy.sharptab.params.lapse_rate(prof, lower = 880., upper =870.)
    lapse4 = sharppy.sharptab.params.lapse_rate(prof, lower = 870., upper =860.)
    lapse5 = sharppy.sharptab.params.lapse_rate(prof, lower = 860., upper =850.)
    lapse6 = sharppy.sharptab.params.lapse_rate(prof, lower = 850., upper =840.)
    lapse7 = sharppy.sharptab.params.lapse_rate(prof, lower = 840., upper =830.)
    lapse8 = sharppy.sharptab.params.lapse_rate(prof, lower = 830., upper =820.)
    lapse9 = sharppy.sharptab.params.lapse_rate(prof, lower = 820., upper =810.)
    lapse10 = sharppy.sharptab.params.lapse_rate(prof, lower = 810., upper =800.)
    lapse11 = sharppy.sharptab.params.lapse_rate(prof, lower = 800., upper =790.)
    lapse12 = sharppy.sharptab.params.lapse_rate(prof, lower = 790., upper =780.)
    lapse13 = sharppy.sharptab.params.lapse_rate(prof, lower = 780., upper =770.)
    lapse14 = sharppy.sharptab.params.lapse_rate(prof, lower = 770., upper =760.)
    lapse15 = sharppy.sharptab.params.lapse_rate(prof, lower = 760., upper =750.)
    lapse16 = sharppy.sharptab.params.lapse_rate(prof, lower = 750., upper =740.)
    lapse17 = sharppy.sharptab.params.lapse_rate(prof, lower = 740., upper =730.)
    lapse18 = sharppy.sharptab.params.lapse_rate(prof, lower = 730., upper =720.)
    lapse19 = sharppy.sharptab.params.lapse_rate(prof, lower = 720., upper =710.)
    lapse20 = sharppy.sharptab.params.lapse_rate(prof, lower = 710., upper =700.)
    lapse21 = sharppy.sharptab.params.lapse_rate(prof, lower = 700., upper =690.)
    lapse22 = sharppy.sharptab.params.lapse_rate(prof, lower = 690., upper =680.)
    lapse23 = sharppy.sharptab.params.lapse_rate(prof, lower = 680., upper =670.)
    lapse24 = sharppy.sharptab.params.lapse_rate(prof, lower = 670., upper =660.)
    lapse25 = sharppy.sharptab.params.lapse_rate(prof, lower = 660., upper =650.)
    lapse26 = sharppy.sharptab.params.lapse_rate(prof, lower = 650., upper =640.)
    lapse27 = sharppy.sharptab.params.lapse_rate(prof, lower = 640., upper =630.)
    lapse28 = sharppy.sharptab.params.lapse_rate(prof, lower = 630., upper =620.)
    lapse29 = sharppy.sharptab.params.lapse_rate(prof, lower = 620., upper =610.)
    lapse30 = sharppy.sharptab.params.lapse_rate(prof, lower = 610., upper =600.)

    hght900 = interp.hght(prof, 900.)
    hght890 = interp.hght(prof, 890.)
    hght880 = interp.hght(prof, 880.)
    hght870 = interp.hght(prof, 870.)
    hght860 = interp.hght(prof, 860.)
    hght850 = interp.hght(prof, 850.)
    hght840 = interp.hght(prof, 840.)
    hght830 = interp.hght(prof, 830.)
    hght820 = interp.hght(prof, 820.)
    hght810 = interp.hght(prof, 810.)
    hght800 = interp.hght(prof, 800.)
    hght790 = interp.hght(prof, 790.)
    hght780 = interp.hght(prof, 780.)
    hght770 = interp.hght(prof, 770.)
    hght760 = interp.hght(prof, 760.)
    hght750 = interp.hght(prof, 750.)
    hght740 = interp.hght(prof, 740.)
    hght730 = interp.hght(prof, 730.)
    hght720 = interp.hght(prof, 720.)
    hght710 = interp.hght(prof, 710.)
    hght700 = interp.hght(prof, 700.)
    hght690 = interp.hght(prof, 690.)
    hght680 = interp.hght(prof, 680.)
    hght670 = interp.hght(prof, 670.)
    hght660 = interp.hght(prof, 660.)
    hght650 = interp.hght(prof, 650.)
    hght640 = interp.hght(prof, 640.)
    hght630 = interp.hght(prof, 630.)
    hght620 = interp.hght(prof, 620.)
    hght610 = interp.hght(prof, 610.)
    hght600 = interp.hght(prof, 600.)

    lapseArray=np.array([lapse1,lapse2,lapse3,lapse4,lapse5,lapse6,lapse7,lapse8,lapse9,lapse10,
                        lapse11,lapse12,lapse13,lapse14,lapse15,lapse16,lapse17,lapse18,lapse19,lapse20,
                        lapse21,lapse22,lapse23,lapse24,lapse25,lapse26,lapse27,lapse28,lapse29,lapse30])

    hghtArray=np.array([hght890,hght880,hght870,hght860,hght850,hght840,hght830,hght820,hght810,hght800,
                       hght790,hght780,hght770,hght760,hght750,hght740,hght730,hght720,hght710,hght700,
                       hght690,hght680,hght670,hght660,hght650,hght640,hght630,hght620,hght610,hght600,])
   
    ESL25 = np.asarray(np.where(lapseArray < 2.5))
    if len(ESL25[0]) != 0:
        ESL25hgtArray = hghtArray[ESL25[0][0]:ESL25[0][-1]]
        esl_25_depth = hghtArray[ESL25[0][-1]] - hghtArray[ESL25[0][0]]
    else:
        esl_25_depth = 0
    
    ESL40 = np.asarray(np.where(lapseArray < 4.0))
    if len(ESL40[0]) != 0:
        ESL40hgtArray = hghtArray[ESL40[0][0]:ESL40[0][-1]]
        esl_40_depth = hghtArray[ESL40[0][-1]] - hghtArray[ESL40[0][0]]
    else:
        esl_40_depth = 0
    #################################################################################################################
    #MOISTURE
    #################################################################################################################
    # Calculate MRD for each parcel type using existing variables
    sb_MRD1 = get_layer(P_sounding, T_sounding, Td_sounding, sb_profile, RH_skewt, bottom = P_sounding[0], depth = P_sounding[0] - sblcl_p)
    ml_MRD1 = get_layer(P_sounding, T_sounding, Td_sounding, ml_profile, RH_skewt, bottom = ml_p, depth = ml_p - mllcl_p)
    mu_MRD1 = get_layer(P_sounding, T_sounding, Td_sounding, mu_profile, RH_skewt, bottom = mu_p, depth = mu_p - mulcl_p)

    #lcl to  lfc+1500m
    sb_MRD2 = get_layer(P_sounding, T_sounding, Td_sounding, sb_profile[::-1], RH_skewt, bottom = sblcl_p, depth = sblcl_p - sblfc1500_p)
    ml_MRD2 = get_layer(P_sounding, T_sounding, Td_sounding, ml_profile[::-1], RH_skewt, bottom = mllcl_p, depth = mllcl_p - mllfc1500_p)
    mu_MRD2 = get_layer(P_sounding, T_sounding, Td_sounding, mu_profile[::-1], RH_skewt, bottom = mulcl_p, depth = mulcl_p - mulfc1500_p)

    #combine the two arrays above (and remove the duplicate lcl level)
    sb_MRD_p = np.concatenate((sb_MRD1[0][:-1], sb_MRD2[0]), axis = 0) 
    sb_MRD_T = np.concatenate((sb_MRD1[1][:-1], sb_MRD2[1]), axis = 0)
    sb_MRD_Td = np.concatenate((sb_MRD1[2][:-1], sb_MRD2[2]), axis = 0)
    sb_MRD_pT = np.concatenate((sb_MRD1[3][:-1], sb_MRD2[3]), axis = 0)
    sb_MRD_RH = np.concatenate((sb_MRD1[4][:-1], sb_MRD2[4]), axis = 0)

    ml_MRD_p = np.concatenate((ml_MRD1[0][:-1], ml_MRD2[0]), axis = 0) 
    ml_MRD_T = np.concatenate((ml_MRD1[1][:-1], ml_MRD2[1]), axis = 0)
    ml_MRD_Td = np.concatenate((ml_MRD1[2][:-1], ml_MRD2[2]), axis = 0)
    ml_MRD_pT = np.concatenate((ml_MRD1[3][:-1], ml_MRD2[3]), axis = 0)
    ml_MRD_RH = np.concatenate((ml_MRD1[4][:-1], ml_MRD2[4]), axis = 0)

    mu_MRD_p = np.concatenate((mu_MRD1[0][:-1], mu_MRD2[0]), axis = 0)
    mu_MRD_T = np.concatenate((mu_MRD1[1][:-1], mu_MRD2[1]), axis = 0)
    mu_MRD_Td = np.concatenate((mu_MRD1[2][:-1], mu_MRD2[2]), axis = 0)
    mu_MRD_pT = np.concatenate((mu_MRD1[3][:-1], mu_MRD2[3]), axis = 0)
    mu_MRD_RH = np.concatenate((mu_MRD1[4][:-1], mu_MRD2[4]), axis = 0)

    #calculate all ml and mu parcel dewpoints & RH's at parcel starting levels 
    # parcel dewpoint = environmental dewpoint from parcel inital height until LCL, environmental temperature from > LCL
    # parcel mixing ratio = environmental mixing ratio from parcel initial height until LCL
    
    sbParcelr = mixing_ratio_from_relative_humidity(P_sounding[0], T_sounding[0], RH_skewt[0])
    mlParcelRH = relative_humidity_from_dewpoint(ml_T, ml_Td)
    mlParcelr = mixing_ratio_from_relative_humidity(ml_p, ml_T, mlParcelRH)
    muParcelRH = relative_humidity_from_dewpoint(mu_T, mu_Td)
    muParcelr = mixing_ratio_from_relative_humidity(mu_p, mu_T, muParcelRH)

    #calculate vertical profile of environmental r
    r_skewt_sb_env = mixing_ratio_from_relative_humidity(sb_MRD_p, sb_MRD_T, sb_MRD_RH)
    r_skewt_ml_env = mixing_ratio_from_relative_humidity(ml_MRD_p, ml_MRD_T, ml_MRD_RH)
    r_skewt_mu_env = mixing_ratio_from_relative_humidity(mu_MRD_p, mu_MRD_T, mu_MRD_RH)
    
    ############
    #1)
    sb_parcel_r1 = np.full(np.shape(sb_MRD1[0][:-1]), sbParcelr)
    ml_parcel_r1 = np.full(np.shape(ml_MRD1[0][:-1]), mlParcelr)
    mu_parcel_r1 = np.full(np.shape(mu_MRD1[0][:-1]), muParcelr)
    
    #2)
    sb_parcel_r2 = mixing_ratio_from_relative_humidity(sb_MRD2[0], sb_MRD2[3], np.full(np.shape(sb_MRD2[0]), 1))
    ml_parcel_r2 = mixing_ratio_from_relative_humidity(ml_MRD2[0], ml_MRD2[3], np.full(np.shape(ml_MRD2[0]), 1))
    mu_parcel_r2 = mixing_ratio_from_relative_humidity(mu_MRD2[0], mu_MRD2[3], np.full(np.shape(mu_MRD2[0]), 1))
    
    #combine arrays to get full parcel r profile between the two levels of interest
    sb_parcel_r = np.concatenate((sb_parcel_r1, sb_parcel_r2.magnitude), axis = 0)
    ml_parcel_r = np.concatenate((ml_parcel_r1, ml_parcel_r2.magnitude), axis = 0)
    mu_parcel_r = np.concatenate((mu_parcel_r1, mu_parcel_r2.magnitude), axis = 0)
    #######
    sbMRD = np.sum(sb_parcel_r - r_skewt_sb_env.magnitude)
    mlMRD = np.sum(ml_parcel_r - r_skewt_ml_env.magnitude)
    muMRD = np.sum(mu_parcel_r - r_skewt_mu_env.magnitude)

    ####################
    pwat = sharppy.sharptab.params.precip_water(prof)
    tei = sharppy.sharptab.params.tei(prof) #theta-e index
    
    #Find Max Theta-E and Q Height in lowest 3 km:
    hgt3km = np.where(hgt_sounding < 3001*units('meter'))
    ThetaE_SkewT = equivalent_potential_temperature(P_sounding, T_sounding, Td_sounding)
    MaxTEvalue = np.max(ThetaE_SkewT[hgt3km])
    ArrayPosition = np.where(ThetaE_SkewT == MaxTEvalue)
    maxTE3kmhgt = hgt_sounding[ArrayPosition[0]]
    MaxQvalue = np.max(Q_skewt[hgt3km])
    ArrayPosition1 = np.where(Q_skewt == MaxQvalue)
    maxQ3kmhgt = hgt_sounding[ArrayPosition1[0]]
    
    meanRH100mb = sharppy.sharptab.params.mean_relh(prof, pbot=sfc, ptop=(sfc-100))
    meanQ100mb = sharppy.sharptab.params.mean_mixratio(prof, pbot=sfc, ptop=(sfc-100))
    meanTe100mb = sharppy.sharptab.params.mean_thetae(prof, pbot=sfc, ptop=(sfc-100))
    meanRH1km = sharppy.sharptab.params.mean_relh(prof, pbot=sfc, ptop=p1km)
    meanQ1km = sharppy.sharptab.params.mean_mixratio(prof, pbot=sfc, ptop=p1km)
    meanTe1km = sharppy.sharptab.params.mean_thetae(prof, pbot=sfc, ptop=p1km)
    meanRH150mb = sharppy.sharptab.params.mean_relh(prof, pbot=sfc, ptop=(sfc-150))
    meanQ150mb = sharppy.sharptab.params.mean_mixratio(prof, pbot=sfc, ptop=(sfc-150))
    meanTe150mb = sharppy.sharptab.params.mean_thetae(prof, pbot=sfc, ptop=(sfc-150))
    meanRH150_300mb = sharppy.sharptab.params.mean_relh(prof, pbot=(sfc-150), ptop=(sfc-300))
    meanQ150_300mb = sharppy.sharptab.params.mean_mixratio(prof, pbot=(sfc-150), ptop=(sfc-300))
    meanTe150_300mb = sharppy.sharptab.params.mean_thetae(prof, pbot=(sfc-150), ptop=(sfc-300))
    meanRHpbl = sharppy.sharptab.params.mean_relh(prof, pbot=sfc, ptop=pbl_top)
    meanQpbl = sharppy.sharptab.params.mean_mixratio(prof, pbot=sfc, ptop=pbl_top)
    meanTepbl = sharppy.sharptab.params.mean_thetae(prof, pbot=sfc, ptop=pbl_top)
    meanRH1_3km = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p3km) 
    meanQ1_3km = sharppy.sharptab.params.mean_mixratio(prof, pbot=p1km, ptop=p3km)
    meanTe1_3km = sharppy.sharptab.params.mean_thetae(prof, pbot=p1km, ptop=p3km)   
    meanRH_EIL = sharppy.sharptab.params.mean_relh(prof,pbot=EILb,ptop=EILt)
    meanQ_EIL = sharppy.sharptab.params.mean_mixratio(prof,pbot=EILb,ptop=EILt)     
    meanTe_EIL = sharppy.sharptab.params.mean_thetae(prof,pbot=EILb,ptop=EILt)
    
    sb_meanRH_LFC_LCL = sharppy.sharptab.params.mean_relh(prof, pbot = sblcl_p.magnitude, ptop=sblfc_p.magnitude)
    ml_meanRH_LFC_LCL = sharppy.sharptab.params.mean_relh(prof, pbot = mllcl_p.magnitude, ptop=mllfc_p.magnitude)
    mu_meanRH_LFC_LCL = sharppy.sharptab.params.mean_relh(prof, pbot = mulcl_p.magnitude, ptop=mulfc_p.magnitude)   
    sb_meanQ_LFC_LCL = sharppy.sharptab.params.mean_mixratio(prof, pbot = sblcl_p.magnitude, ptop=sblfc_p.magnitude)
    ml_meanQ_LFC_LCL = sharppy.sharptab.params.mean_mixratio(prof, pbot = mllcl_p.magnitude, ptop=mllfc_p.magnitude)
    mu_meanQ_LFC_LCL = sharppy.sharptab.params.mean_mixratio(prof, pbot = mulcl_p.magnitude, ptop=mulfc_p.magnitude)
    sb_meanTe_LFC_LCL = sharppy.sharptab.params.mean_thetae(prof, pbot = sblcl_p.magnitude, ptop=sblfc_p.magnitude)
    ml_meanTe_LFC_LCL = sharppy.sharptab.params.mean_thetae(prof, pbot = mllcl_p.magnitude, ptop=mllfc_p.magnitude)
    mu_meanTe_LFC_LCL = sharppy.sharptab.params.mean_thetae(prof, pbot = mulcl_p.magnitude, ptop=mulfc_p.magnitude)
   
    meanRHsb_ACBL = sharppy.sharptab.params.mean_relh(prof, pbot=sblfc_p.magnitude, ptop=sblfc1500_p.magnitude)
    meanRHml_ACBL = sharppy.sharptab.params.mean_relh(prof, pbot=mllfc_p.magnitude, ptop=mllfc1500_p.magnitude)
    meanRHmu_ACBL = sharppy.sharptab.params.mean_relh(prof, pbot=mulfc_p.magnitude, ptop=mulfc1500_p.magnitude)
    meanQsb_ACBL = sharppy.sharptab.params.mean_mixratio(prof, pbot=sblfc_p.magnitude, ptop=sblfc1500_p.magnitude)
    meanQml_ACBL = sharppy.sharptab.params.mean_mixratio(prof, pbot=mllfc_p.magnitude, ptop=mllfc1500_p.magnitude)
    meanQmu_ACBL = sharppy.sharptab.params.mean_mixratio(prof, pbot=mulfc_p.magnitude, ptop=mulfc1500_p.magnitude)
    meanTesb_ACBL = sharppy.sharptab.params.mean_thetae(prof, pbot=sblfc_p.magnitude, ptop=sblfc1500_p.magnitude)
    meanTeml_ACBL = sharppy.sharptab.params.mean_thetae(prof, pbot=mllfc_p.magnitude, ptop=mllfc1500_p.magnitude)   
    meanTemu_ACBL = sharppy.sharptab.params.mean_thetae(prof, pbot=mulfc_p.magnitude, ptop=mulfc1500_p.magnitude)
    
    meanRHsb_LCL2k = sharppy.sharptab.params.mean_relh(prof, pbot=sblcl_p.magnitude, ptop=sblcl2000_p.magnitude)
    meanRHml_LCL2k = sharppy.sharptab.params.mean_relh(prof, pbot=mllcl_p.magnitude, ptop=mllcl2000_p.magnitude)
    meanRHmu_LCL2k = sharppy.sharptab.params.mean_relh(prof, pbot=mulcl_p.magnitude, ptop=mulcl2000_p.magnitude)
    meanQsb_LCL2k = sharppy.sharptab.params.mean_mixratio(prof, pbot=sblcl_p.magnitude, ptop=sblcl2000_p.magnitude)
    meanQml_LCL2k = sharppy.sharptab.params.mean_mixratio(prof, pbot=mllcl_p.magnitude, ptop=mllcl2000_p.magnitude)
    meanQmu_LCL2k = sharppy.sharptab.params.mean_mixratio(prof, pbot=mulcl_p.magnitude, ptop=mulcl2000_p.magnitude) 
    meanTesb_LCL2k = sharppy.sharptab.params.mean_thetae(prof, pbot=sblcl_p.magnitude, ptop=sblcl2000_p.magnitude)    
    meanTeml_LCL2k = sharppy.sharptab.params.mean_thetae(prof, pbot=mllcl_p.magnitude, ptop=mllcl2000_p.magnitude)
    meanTemu_LCL2k = sharppy.sharptab.params.mean_thetae(prof, pbot=mulcl_p.magnitude, ptop=mulcl2000_p.magnitude)
      
    mean500m_3km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p500m, ptop=p3km)
    mean500m_4km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p500m, ptop=p4km)
    mean500m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p500m, ptop=p5km)
    mean500m_6km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p500m, ptop=p6km)
    mean1000m_3km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p3km)
    mean1000m_4km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p4km)
    mean1000m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p5km)
    mean1000m_6km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p6km)
    mean1500m_3km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1500m, ptop=p3km)
    mean1500m_4km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1500m, ptop=p4km)
    mean1500m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1500m, ptop=p5km)
    mean1500m_6km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1500m, ptop=p6km)
    mean2000m_3km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2km, ptop=p3km)
    mean2000m_4km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2km, ptop=p4km)
    mean2000m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2km, ptop=p5km)
    mean2000m_6km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2km, ptop=p6km)
    mean2500m_3km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2500m, ptop=p3km)
    mean2500m_4km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2500m, ptop=p4km)
    mean2500m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2500m, ptop=p5km)
    mean2500m_6km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2500m, ptop=p6km)
    mean3000m_4km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p4km)
    mean3000m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p5km)
    mean3000m_6km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p1km, ptop=p6km)
    #################################################################################################################
    #################################################################################################################
    sbCAPECIN_500m_3km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p3km, flag=1)
    sbCAPECIN_500m_4km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p4km, flag=1)
    sbCAPECIN_500m_5km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p5km, flag=1)
    sbCAPECIN_500m_6km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p6km, flag=1)   
    mlCAPECIN_500m_3km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p3km, flag=4)
    mlCAPECIN_500m_4km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p4km, flag=4)
    mlCAPECIN_500m_5km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p5km, flag=4)
    mlCAPECIN_500m_6km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p6km, flag=4)
    muCAPECIN_500m_3km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p3km, flag=3)
    muCAPECIN_500m_4km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p4km, flag=3)
    muCAPECIN_500m_5km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p5km, flag=3)
    muCAPECIN_500m_6km = sharppy.sharptab.params.cape(prof, pbot=p500m, ptop=p6km, flag=3)
    
    sbCAPECIN_1_3km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p3km, flag=1)
    sbCAPECIN_1_4km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p4km, flag=1)
    sbCAPECIN_1_5km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p5km, flag=1)
    sbCAPECIN_1_6km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p6km, flag=1)   
    mlCAPECIN_1_3km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p3km, flag=4)
    mlCAPECIN_1_4km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p4km, flag=4)
    mlCAPECIN_1_5km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p5km, flag=4)
    mlCAPECIN_1_6km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p6km, flag=4)
    muCAPECIN_1_3km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p3km, flag=3)
    muCAPECIN_1_4km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p4km, flag=3)
    muCAPECIN_1_5km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p5km, flag=3)
    muCAPECIN_1_6km = sharppy.sharptab.params.cape(prof, pbot=p1km, ptop=p6km, flag=3)
    
    sbCAPECIN_1500m_3km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p3km, flag=1)
    sbCAPECIN_1500m_4km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p4km, flag=1)
    sbCAPECIN_1500m_5km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p5km, flag=1)
    sbCAPECIN_1500m_6km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p6km, flag=1)   
    mlCAPECIN_1500m_3km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p3km, flag=4)
    mlCAPECIN_1500m_4km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p4km, flag=4)
    mlCAPECIN_1500m_5km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p5km, flag=4)
    mlCAPECIN_1500m_6km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p6km, flag=4)
    muCAPECIN_1500m_3km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p3km, flag=3)
    muCAPECIN_1500m_4km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p4km, flag=3)
    muCAPECIN_1500m_5km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p5km, flag=3)
    muCAPECIN_1500m_6km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p6km, flag=3)

    sbCAPECIN_2_3km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p3km, flag=1)
    sbCAPECIN_2_4km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p4km, flag=1)
    sbCAPECIN_2_5km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p5km, flag=1)
    sbCAPECIN_2_6km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p6km, flag=1)   
    mlCAPECIN_2_3km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p3km, flag=4)
    mlCAPECIN_2_4km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p4km, flag=4)
    mlCAPECIN_2_5km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p5km, flag=4)
    mlCAPECIN_2_6km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p6km, flag=4)
    muCAPECIN_2_3km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p3km, flag=3)
    muCAPECIN_2_4km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p4km, flag=3)
    muCAPECIN_2_5km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p5km, flag=3)
    muCAPECIN_2_6km = sharppy.sharptab.params.cape(prof, pbot=p2km, ptop=p6km, flag=3)    
    #################################################################################################################
    # BUOYANCY
    #################################################################################################################
    mlcape = mlpcl.bplus
    mucape =  mupcl.bplus
    sbcape = sbpcl.bplus
    
    sb_LCLCAPE1 = sharppy.sharptab.params.cape(prof, pbot= float(sblcl_p.magnitude), ptop= float(sblcl2000_p.magnitude),
                                               flag = 1, pres = float(P_sounding[0].magnitude), 
                                               tmpc = float(T_sounding[0].magnitude), dwpc = float(Td_sounding[0].magnitude))
    ml_LCLCAPE1 = sharppy.sharptab.params.cape(prof, pbot= float(mllcl_p.magnitude), ptop= float(mllcl2000_p.magnitude),
                                               flag = 4, pres = float(ml_p.magnitude), 
                                               tmpc = float(ml_T.magnitude), dwpc = float(ml_Td.magnitude))
    mu_LCLCAPE1 = sharppy.sharptab.params.cape(prof, pbot= float(mulcl_p.magnitude), ptop= float(mulcl2000_p.magnitude),
                                               flag = 3, pres = float(mu_p.magnitude),
                                               tmpc = float(mu_T.magnitude), dwpc = float(mu_Td.magnitude))
    sb_LCLCAPE = sb_LCLCAPE1.bplus + sb_LCLCAPE1.bminus
    ml_LCLCAPE = ml_LCLCAPE1.bplus + ml_LCLCAPE1.bminus
    mu_LCLCAPE = mu_LCLCAPE1.bplus + mu_LCLCAPE1.bminus
   
    sb3kmCAPE = sbpcl.b3km
    ml3kmCAPE = mlpcl.b3km
    mu3kmCAPE = mupcl.b3km

    sb6kmCAPE = sbpcl.b6km
    ml6kmCAPE = mlpcl.b6km
    mu6kmCAPE = mupcl.b6km

    sbfrzCAPE = sbpcl.bfzl
    mlfrzCAPE = mlpcl.bfzl
    mufrzCAPE = mupcl.bfzl

    dCAPE = sharppy.sharptab.params.dcape(prof)
    
    sb_limax = sbpcl.limax
    ml_limax = mlpcl.limax
    mu_limax = mupcl.limax
    
    ecape = calc_ecape(msl_sounding, P_sounding, T_sounding, Q_skewt, u_sounding, v_sounding)
    
    sbCAPE_500m_3km = sbCAPECIN_500m_3km.bplus
    sbCAPE_500m_4km = sbCAPECIN_500m_4km.bplus
    sbCAPE_500m_5km = sbCAPECIN_500m_5km.bplus
    sbCAPE_500m_6km = sbCAPECIN_500m_6km.bplus
    mlCAPE_500m_3km = mlCAPECIN_500m_3km.bplus
    mlCAPE_500m_4km = mlCAPECIN_500m_4km.bplus
    mlCAPE_500m_5km = mlCAPECIN_500m_5km.bplus
    mlCAPE_500m_6km = mlCAPECIN_500m_6km.bplus
    muCAPE_500m_3km = muCAPECIN_500m_3km.bplus
    muCAPE_500m_4km = muCAPECIN_500m_4km.bplus
    muCAPE_500m_5km = muCAPECIN_500m_5km.bplus
    muCAPE_500m_6km = muCAPECIN_500m_6km.bplus
    
    sbCAPE_1_3km = sbCAPECIN_1_3km.bplus
    sbCAPE_1_4km = sbCAPECIN_1_4km.bplus
    sbCAPE_1_5km = sbCAPECIN_1_5km.bplus
    sbCAPE_1_6km = sbCAPECIN_1_6km.bplus   
    mlCAPE_1_3km = mlCAPECIN_1_3km.bplus
    mlCAPE_1_4km = mlCAPECIN_1_4km.bplus
    mlCAPE_1_5km = mlCAPECIN_1_5km.bplus
    mlCAPE_1_6km = mlCAPECIN_1_6km.bplus
    muCAPE_1_3km = muCAPECIN_1_3km.bplus
    muCAPE_1_4km = muCAPECIN_1_4km.bplus
    muCAPE_1_5km = muCAPECIN_1_5km.bplus
    muCAPE_1_6km = muCAPECIN_1_6km.bplus

    sbCAPE_1500m_3km = sbCAPECIN_1500m_3km.bplus
    sbCAPE_1500m_4km = sbCAPECIN_1500m_4km.bplus
    sbCAPE_1500m_5km = sbCAPECIN_1500m_5km.bplus
    sbCAPE_1500m_6km = sbCAPECIN_1500m_6km.bplus
    mlCAPE_1500m_3km = mlCAPECIN_1500m_3km.bplus
    mlCAPE_1500m_4km = mlCAPECIN_1500m_4km.bplus
    mlCAPE_1500m_5km = mlCAPECIN_1500m_5km.bplus
    mlCAPE_1500m_6km = mlCAPECIN_1500m_6km.bplus
    muCAPE_1500m_3km = muCAPECIN_1500m_3km.bplus
    muCAPE_1500m_4km = muCAPECIN_1500m_4km.bplus
    muCAPE_1500m_5km = muCAPECIN_1500m_5km.bplus
    muCAPE_1500m_6km = muCAPECIN_1500m_6km.bplus
    
    sbCAPE_2_3km = sbCAPECIN_2_3km.bplus
    sbCAPE_2_4km = sbCAPECIN_2_4km.bplus
    sbCAPE_2_5km = sbCAPECIN_2_5km.bplus
    sbCAPE_2_6km = sbCAPECIN_2_6km.bplus   
    mlCAPE_2_3km = mlCAPECIN_2_3km.bplus
    mlCAPE_2_4km = mlCAPECIN_2_4km.bplus
    mlCAPE_2_5km = mlCAPECIN_2_5km.bplus
    mlCAPE_2_6km = mlCAPECIN_2_6km.bplus
    muCAPE_2_3km = muCAPECIN_2_3km.bplus
    muCAPE_2_4km = muCAPECIN_2_4km.bplus
    muCAPE_2_5km = muCAPECIN_2_5km.bplus
    muCAPE_2_6km = muCAPECIN_2_6km.bplus
    #################################################################################################################
    #INHIBITION
    #################################################################################################################
    mlcin = mlpcl.bminus
    mucin =  mupcl.bminus
    sbcin = sbpcl.bminus
    
    ConvT = sharppy.sharptab.params.convective_temp(prof) #degC
    T_700 = interp.temp(prof, 700.)          
    
    sbCIN_500m_3km = sbCAPECIN_500m_3km.bminus
    sbCIN_500m_4km = sbCAPECIN_500m_4km.bminus
    sbCIN_500m_5km = sbCAPECIN_500m_5km.bminus
    sbCIN_500m_6km = sbCAPECIN_500m_6km.bminus
    mlCIN_500m_3km = mlCAPECIN_500m_3km.bminus
    mlCIN_500m_4km = mlCAPECIN_500m_4km.bminus
    mlCIN_500m_5km = mlCAPECIN_500m_5km.bminus
    mlCIN_500m_6km = mlCAPECIN_500m_6km.bminus
    muCIN_500m_3km = muCAPECIN_500m_3km.bminus
    muCIN_500m_4km = muCAPECIN_500m_4km.bminus
    muCIN_500m_5km = muCAPECIN_500m_5km.bminus
    muCIN_500m_6km = muCAPECIN_500m_6km.bminus
    
    sbCIN_1_3km = sbCAPECIN_1_3km.bminus
    sbCIN_1_4km = sbCAPECIN_1_4km.bminus
    sbCIN_1_5km = sbCAPECIN_1_5km.bminus
    sbCIN_1_6km = sbCAPECIN_1_6km.bminus   
    mlCIN_1_3km = mlCAPECIN_1_3km.bminus
    mlCIN_1_4km = mlCAPECIN_1_4km.bminus
    mlCIN_1_5km = mlCAPECIN_1_5km.bminus
    mlCIN_1_6km = mlCAPECIN_1_6km.bminus
    muCIN_1_3km = muCAPECIN_1_3km.bminus
    muCIN_1_4km = muCAPECIN_1_4km.bminus
    muCIN_1_5km = muCAPECIN_1_5km.bminus
    muCIN_1_6km = muCAPECIN_1_6km.bminus

    sbCIN_1500m_3km = sbCAPECIN_1500m_3km.bminus
    sbCIN_1500m_4km = sbCAPECIN_1500m_4km.bminus
    sbCIN_1500m_5km = sbCAPECIN_1500m_5km.bminus
    sbCIN_1500m_6km = sbCAPECIN_1500m_6km.bminus
    mlCIN_1500m_3km = mlCAPECIN_1500m_3km.bminus
    mlCIN_1500m_4km = mlCAPECIN_1500m_4km.bminus
    mlCIN_1500m_5km = mlCAPECIN_1500m_5km.bminus
    mlCIN_1500m_6km = mlCAPECIN_1500m_6km.bminus
    muCIN_1500m_3km = muCAPECIN_1500m_3km.bminus
    muCIN_1500m_4km = muCAPECIN_1500m_4km.bminus
    muCIN_1500m_5km = muCAPECIN_1500m_5km.bminus
    muCIN_1500m_6km = muCAPECIN_1500m_6km.bminus
    
    sbCIN_2_3km = sbCAPECIN_2_3km.bminus
    sbCIN_2_4km = sbCAPECIN_2_4km.bminus
    sbCIN_2_5km = sbCAPECIN_2_5km.bminus
    sbCIN_2_6km = sbCAPECIN_2_6km.bminus   
    mlCIN_2_3km = mlCAPECIN_2_3km.bminus
    mlCIN_2_4km = mlCAPECIN_2_4km.bminus
    mlCIN_2_5km = mlCAPECIN_2_5km.bminus
    mlCIN_2_6km = mlCAPECIN_2_6km.bminus
    muCIN_2_3km = muCAPECIN_2_3km.bminus
    muCIN_2_4km = muCAPECIN_2_4km.bminus
    muCIN_2_5km = muCAPECIN_2_5km.bminus
    muCIN_2_6km = muCAPECIN_2_6km.bminus
    #################################################################################################################
    # MOTION
    #################################################################################################################
    #calculate storm motion using Bunkers et. al 2014
    #using effective inflow base as base, and pressure weighted mean wind    
    srwind = params.bunkers_storm_motion(prof)
    #this gives us u and v motion vectors for right movers [0 and 1] and left movers [2 and 3]

    bunkersU = srwind[0]
    bunkersV = srwind[1]
    vectorMag = math.sqrt(((bunkersU)**2) + ((bunkersV)**2))
    angle = math.degrees(math.atan2(bunkersV,bunkersU)) #angle from X axis
    #################################################################################################################
    #SRH Calculations
    #################################################################################################################
    srh500m = winds.helicity(prof, 0, 500., stu = srwind[0], stv = srwind[1])[0]
    srh1km = winds.helicity(prof, 0, 1000., stu = srwind[0], stv = srwind[1])[0]
    srh3km = winds.helicity(prof, 0, 3000., stu = srwind[0], stv = srwind[1])[0]
    effective_srh = winds.helicity(prof, ebot_hght, etop_hght, stu = srwind[0], stv = srwind[1])[0]
    #################################################################################################################
    #BULK SHEAR
    #################################################################################################################
    #Get shear magnitudes
    ml_halfELpres = interp.pres(prof, interp.to_msl(prof, mlpcl.elhght/2.))
    mu_halfELpres = interp.pres(prof, interp.to_msl(prof, mupcl.elhght/2.))
    sb_halfELpres = interp.pres(prof, interp.to_msl(prof, sbpcl.elhght/2.))

    ml_ebwd = winds.wind_shear(prof, pbot=EILb, ptop=ml_halfELpres)
    ml_ebwspd = utils.mag(ml_ebwd[0], ml_ebwd[1])
    mu_ebwd = winds.wind_shear(prof, pbot=EILb, ptop=mu_halfELpres)
    mu_ebwspd = utils.mag(mu_ebwd[0], mu_ebwd[1])
    sb_ebwd = winds.wind_shear(prof, pbot=EILb, ptop=sb_halfELpres)
    sb_ebwspd = utils.mag(sb_ebwd[0], sb_ebwd[1])
    
    sfc_500m_shear = winds.wind_shear(prof, pbot=sfc, ptop=p500m)
    sfc500mshear = utils.mag(sfc_500m_shear[0], sfc_500m_shear[1])
    sfc_1km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p1km)
    sfc1shear = utils.mag(sfc_1km_shear[0], sfc_1km_shear[1])
    sfc_2km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p2km)
    sfc2shear = utils.mag(sfc_2km_shear[0], sfc_2km_shear[1])
    sfc_3km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p3km)
    sfc3shear = utils.mag(sfc_3km_shear[0], sfc_3km_shear[1])
    sfc_5km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p5km)
    sfc5shear = utils.mag(sfc_5km_shear[0], sfc_5km_shear[1]) 
    sfc_6km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p6km)
    sfc6shear = utils.mag(sfc_6km_shear[0], sfc_6km_shear[1])
    sfc_8km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p8km)
    sfc8shear = utils.mag(sfc_8km_shear[0], sfc_8km_shear[1])
    EIL_shear = winds.wind_shear(prof, pbot=EILb, ptop=EILt)
    EILshear = utils.mag(EIL_shear[0], EIL_shear[1])

    sb_lcl_el_shear = winds.wind_shear(prof, pbot=sbpcl.lclpres, ptop = sbpcl.elpres)
    ml_lcl_el_shear = winds.wind_shear(prof, pbot=mlpcl.lclpres, ptop = mlpcl.elpres)
    mu_lcl_el_shear = winds.wind_shear(prof, pbot=mupcl.lclpres, ptop = mupcl.elpres)
    sb_lclELshear = utils.mag(sb_lcl_el_shear[0],sb_lcl_el_shear[1])
    ml_lclELshear = utils.mag(ml_lcl_el_shear[0],ml_lcl_el_shear[1])
    mu_lclELshear = utils.mag(mu_lcl_el_shear[0],mu_lcl_el_shear[1]) 
    #################################################################################################################
    #STORM RELATIVE FLOW
    #################################################################################################################
    #calculate u and v storm relative wind vectors for layers of interest, using Bunkers Right Moving u and v storm motion vectors
    srflow0500m_vector = winds.sr_wind(prof, pbot = sfc, ptop = p500m, stu = srwind[0], stv = srwind[1])
    srflow01km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p1km, stu = srwind[0], stv = srwind[1])
    srflow02km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p2km, stu = srwind[0], stv = srwind[1])
    srflow03km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p3km, stu = srwind[0], stv = srwind[1])
    srflow06km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p6km, stu = srwind[0], stv = srwind[1])
    srflow08km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p8km, stu = srwind[0], stv = srwind[1])
    srflow4_5km_vector = winds.sr_wind(prof, pbot = p4km, ptop = p5km, stu = srwind[0], stv = srwind[1])
    srflow4_6km_vector = winds.sr_wind(prof, pbot = p4km, ptop = p6km, stu = srwind[0], stv = srwind[1])
    sb_srflowLCL_EL_vector = winds.sr_wind(prof, pbot = sbpcl.lclpres, ptop = sbpcl.elpres, stu = srwind[0], stv = srwind[1])
    ml_srflowLCL_EL_vector = winds.sr_wind(prof, pbot = mlpcl.lclpres, ptop = mlpcl.elpres, stu = srwind[0], stv = srwind[1])
    mu_srflowLCL_EL_vector = winds.sr_wind(prof, pbot = mupcl.lclpres, ptop = mupcl.elpres, stu = srwind[0], stv = srwind[1])    
    srflowEIL_vector = winds.sr_wind(prof, pbot = EILb, ptop = EILt, stu = srwind[0], stv = srwind[1])
    
    srflow0500m = utils.mag(srflow0500m_vector[0], srflow0500m_vector[1])
    srflow01km = utils.mag(srflow01km_vector[0], srflow01km_vector[1])
    srflow02km = utils.mag(srflow02km_vector[0], srflow02km_vector[1])
    srflow03km = utils.mag(srflow03km_vector[0], srflow03km_vector[1])    
    srflow06km = utils.mag(srflow06km_vector[0], srflow06km_vector[1])
    srflow08km = utils.mag(srflow08km_vector[0], srflow08km_vector[1])
    srflow4_5km = utils.mag(srflow4_5km_vector[0], srflow4_5km_vector[1])
    srflow4_6km = utils.mag(srflow4_6km_vector[0], srflow4_6km_vector[1])
    sb_srflowLCL_EL = utils.mag(sb_srflowLCL_EL_vector[0], sb_srflowLCL_EL_vector[1])
    ml_srflowLCL_EL = utils.mag(ml_srflowLCL_EL_vector[0], ml_srflowLCL_EL_vector[1])
    mu_srflowLCL_EL = utils.mag(mu_srflowLCL_EL_vector[0], mu_srflowLCL_EL_vector[1])
    srflowEIL = utils.mag(srflowEIL_vector[0], srflowEIL_vector[1])    
    #################################################################################################################
    #OTHER SHEAR
    #################################################################################################################
    #subcloud shear (subcloud = parcel inital level to parcel's LCL)
    sb_subcloud_shear = bulk_shear(P_sounding, u_sounding, v_sounding, depth = P_sounding[0] - sblcl_p, bottom = P_sounding[0])
    ml_subcloud_shear = bulk_shear(P_sounding, u_sounding, v_sounding, depth = ml_p - mllcl_p, bottom = ml_p)
    mu_subcloud_shear = bulk_shear(P_sounding, u_sounding, v_sounding, depth = mu_p - mulcl_p, bottom = mu_p)
    sb_SC_shear = (ma.sqrt(sb_subcloud_shear[0]**2 + sb_subcloud_shear[1]**2))/(sblcl_h.magnitude)
    ml_SC_shear = (ma.sqrt(ml_subcloud_shear[0]**2 + ml_subcloud_shear[1]**2))/(mllcl_h.magnitude)
    mu_SC_shear = (ma.sqrt(mu_subcloud_shear[0]**2 + mu_subcloud_shear[1]**2))/(mulcl_h.magnitude)
    
    # ACBL Shear    
    sb_ACBL_shear = bulk_shear(P_sounding, u_sounding, v_sounding, depth = sblfc_p - sblfc1500_p, bottom = sblfc_p)
    ml_ACBL_shear = bulk_shear(P_sounding, u_sounding, v_sounding, depth = mllfc_p - mllfc1500_p, bottom = mllfc_p)
    mu_ACBL_shear = bulk_shear(P_sounding, u_sounding, v_sounding, depth = mulfc_p - mulfc1500_p, bottom = mulfc_p)
    sb_acbl_shear = (ma.sqrt(sb_ACBL_shear[0]**2 + sb_ACBL_shear[1]**2))/((sblfc_h1500-sblfc_h).magnitude)
    ml_acbl_shear = (ma.sqrt(ml_ACBL_shear[0]**2 + ml_ACBL_shear[1]**2))/((mllfc_h1500-mllfc_h).magnitude)
    mu_acbl_shear = (ma.sqrt(mu_ACBL_shear[0]**2 + mu_ACBL_shear[1]**2))/((mulfc_h1500-mulfc_h).magnitude)

    critAng = sharppy.sharptab.winds.critical_angle(prof)
    
    mu_brnSHR = mupcl.brnshear
    sb_brnSHR = sbpcl.brnshear
    if isinstance(sb_brnSHR, np.ma.core.MaskedConstant):
        sb_brnSHR = mu_brnSHR
    ml_brnSHR = mlpcl.brnshear
    if isinstance(ml_brnSHR, np.ma.core.MaskedConstant):
        ml_brnSHR = mu_brnSHR

    u_1=u_sounding.magnitude/1.944
    v_1=v_sounding.magnitude/1.944
    z_1=hgt_sounding.magnitude
    
    profMS = profile.create_profile(profile='default', pres=P_sounding, hght=hgt_sounding, tmpc=T_sounding,
                                      dwpc=Td_sounding, u=u_1, v=v_1, missing=-9999)
    
    srwindMS = params.bunkers_storm_motion(profMS) # u and v motion vectors for RM [0 and 1] and LM [2 and 3]
    
    uSR_W = u_1-srwindMS[0]
    vSR_W = v_1-srwindMS[1]
    
    dudz_W=(np.array(u_1[2::])-np.array(u_1[0:-2]))/(np.array(z_1[2::])-np.array(z_1[0:-2]))
    dvdz_W=(np.array(v_1[2::])-np.array(v_1[0:-2]))/(np.array(z_1[2::])-np.array(z_1[0:-2]))
    dudz_W=np.insert(dudz_W,0,dudz_W[0])
    dudz_W=np.insert(dudz_W,-1,dudz_W[-1])
    dvdz_W=np.insert(dvdz_W,0,dvdz_W[0])
    dvdz_W=np.insert(dvdz_W,-1,dvdz_W[-1])

    shear_W=np.sqrt(dudz_W**2+dvdz_W**2)+0.0000001
    # Vorticity components
    uvort_W=-dvdz_W
    vvort_W=dudz_W
    # Streamwise vorticity
    strmw_W=abs((uSR_W*uvort_W+vSR_W*vvort_W)/(np.sqrt(uSR_W**2+vSR_W**2)))
    # Streamwise fraction
    strmwperc_W=strmw_W/shear_W
    
    hgt500m = np.where(hgt_sounding < 501*units('meter'))
    hgt1km = np.where(hgt_sounding < 1001*units('meter'))
    hgt3km = np.where(hgt_sounding < 3001*units('meter')) 
    MeanShearValue_500m = np.mean(shear_W[hgt500m])
    MeanSWvalue_500m = np.mean(strmw_W[hgt500m])
    MeanSWpercvalue_500m = np.mean(strmwperc_W[hgt500m])
    MeanShearValue_1km = np.mean(shear_W[hgt1km])
    MeanSWvalue_1km = np.mean(strmw_W[hgt1km])
    MeanSWpercvalue_1km = np.mean(strmwperc_W[hgt1km])
    MeanShearValue_3km = np.mean(shear_W[hgt3km])
    MeanSWvalue_3km = np.mean(strmw_W[hgt3km])
    MeanSWpercvalue_3km = np.mean(strmwperc_W[hgt3km])  

    hgt1_3km = np.where((hgt_sounding > 1001*units('meter')) & (hgt_sounding < 3001*units('meter')))
    MeanShearValue1_3km = np.mean(shear_W[hgt1_3km])
    MeanSWvalue1_3km = np.mean(strmw_W[hgt1_3km])
    MeanSWpercvalue1_3km = np.mean(strmwperc_W[hgt1_3km])  
    #################################################################################################################
    # COMPOSITES    
    #################################################################################################################
    SCP = scp(mupcl.bplus, effective_srh, mu_ebwspd, mupcl.bminus)
    STP_FL = stp_fixed(sbpcl.bplus, sbpcl.lclhght, srh1km, sfc6shear, sbpcl.bminus)
    STP_EL = stp_effective(mlpcl.bplus, effective_srh, ml_ebwspd, mlpcl.lclhght, mlpcl.bminus, ebot_hght)

    if etop_hght > 500.0:
        srh500mEIL = srh500m  # Entire 0–500m layer is within the EIL
    else:
        srh500mEIL = effective_srh  # Only use the SRH within the EIL

    STP_500 = stp_500mSRH(mlpcl.bplus, mlpcl.lclhght, srh500mEIL, ml_ebwspd, mlpcl.bminus, ebot_hght, etop_hght)
    VTP = vtp(mlpcl.bplus, mlpcl.lclhght, effective_srh, ml_ebwspd, mlpcl.bminus, mlpcl.b3km, lapse0_3km, ebot_hght)

    sbBRN = sbcape/sb_brnSHR
    mlBRN = mlcape/ml_brnSHR
    muBRN = mucape/mu_brnSHR
    
    #Mburst = sharppy.sharptab.params.mburst(prof)
    dcp = sharppy.sharptab.params.dcp(prof)
    sb_0_1ehi = sharppy.sharptab.params.ehi(prof, sbpcl, hbot= 0, htop = 1000) 
    sb_0_3ehi = sharppy.sharptab.params.ehi(prof, sbpcl, hbot= 0, htop = 3000)
    ml_0_1ehi = sharppy.sharptab.params.ehi(prof, mlpcl, hbot= 0, htop = 1000) 
    ml_0_3ehi = sharppy.sharptab.params.ehi(prof, mlpcl, hbot= 0, htop = 3000)
    mu_0_1ehi = sharppy.sharptab.params.ehi(prof, mupcl, hbot= 0, htop = 1000)
    mu_0_3ehi = sharppy.sharptab.params.ehi(prof, mupcl, hbot= 0, htop = 3000)
    wndg = sharppy.sharptab.params.wndg(prof)
    #mmp = sharppy.sharptab.params.mmp(prof)
    ship = sharppy.sharptab.params.ship(prof)
    sig_sev = sharppy.sharptab.params.sig_severe(prof)
    #sweat = sharppy.sharptab.params.sweat(prof)
    #################################################################################################################  
    #################################################################################################################
    #Quality Control Statement to Flag for convective contamination (if RH > 90% in all layers between 0.5 and 6 km)
    RH_QC_layer = get_layer(P_sounding, RH_skewt, bottom = p500m*units('hectopascal'), depth = (5500*units('meter')))
    RH_QC_list = RH_QC_layer[1].tolist()
    if mucape > 0:
        QC = all(i > 0.90 for i in RH_QC_list)
    else:
        QC = True
        
    ################################################## OUTPUTS ######################################################
    #create a .txt file that can be loaded into a sounding viewer script
    #SoundingOut = [P_sounding.magnitude, T_sounding.magnitude, Td_sounding.magnitude, hgt_sounding.magnitude, u_sounding.magnitude, v_sounding.magnitude]
    #dfSounding = pd.DataFrame(SoundingOut)
    #dfSounding2 = dfSounding.T
    
    new_row = {       
    'StormID':StormID,
    'Profile_Number':prof_num,
    'Time':hour,
    'Watch':dfread['Watch'].astype("int64"),
    'Grouping':dfread['Grouping'].astype('str'),
    'Radar':dfread['Radar'].astype('str'),
    'Lat':dfread['Lat'].astype("int64"),
    'Lon':dfread['Lon'].astype("int64"),
    'Type':dfread['Type'].astype('str'),
    'Discrete':dfread['Discrete'].astype('str'),
    'Supercell':dfread['Supercell'].astype('str'),
    'LeftSplit':dfread['LeftSplit'].astype('str'),
    'Severe30':dfread['Severe30'].astype('str'),
    'ConfirmedTor30':dfread['ConfirmedTor30'].astype('str'),
    'Profile':dfread['Profile'].astype("int64"),
    'MaxProfiles':dfread['MaxProfiles'].astype("int64"),
    'Daylight':dfread['Daylight'].astype('str'),
    'SupercellAny':dfread['SupercellAny'].astype('str'),
    'SevereAny':dfread['SevereAny'].astype('str'),
    'TorAny':dfread['TorAny'].astype('str'),
    'Start':dfread['Start'].astype('str'),
    'End':dfread['End'].astype('str'),
    'Life (in mins)':dfread['Life (in mins)'].astype("int64"),
    'Demise':dfread['Demise'].astype('str'),
    'Ini_Lat':dfread['Ini_Lat'].astype('str'),
    'Ini_Lon':dfread['Ini_Lon'].astype('str'),  
    'cenlat':cenlat,
    'cenlon':cenlon,
    'ModelLat':lat[x][y],
    'ModelLon':lon[x][y],
    'Prof_QC_issue_(bool)':QC,

    'sbLCL':sblcl_h.magnitude,
    'mlLCL':mllcl_h.magnitude,
    'muLCL':mulcl_h.magnitude,
    'sbLFC':sblfc_h.magnitude,
    'mlLFC':mllfc_h.magnitude,
    'muLFC':mulfc_h.magnitude,
    'sb_LFC_LCL_diff':sb_LFC_LCL_diff.magnitude,
    'ml_LFC_LCL_diff':ml_LFC_LCL_diff.magnitude,
    'mu_LFC_LCL_diff':mu_LFC_LCL_diff.magnitude,
    'pbl_top':pbl_top,
    'EIL_depth':EIL_depth,
    'EIL_PBL_hgtdiff':EIL_PBL_hgtdiff.magnitude,
    'sbMaxOmegaH':sbMaxOmegaH.magnitude,
    'mlMaxOmegaH':mlMaxOmegaH.magnitude,
    'muMaxOmegaH':muMaxOmegaH.magnitude,
    'sbHLFC':sbHLFC[0],
    'mlHLFC':mlHLFC[0],
    'muHLFC':muHLFC[0],
    'deltaZstar':deltaZstar,

    'sbMaxOmega':sbMaxOmega.magnitude,
    'mlMaxOmega':mlMaxOmega.magnitude,
    'muMaxOmega':muMaxOmega.magnitude,
    'sbSCconv':sb_SCconv.magnitude,
    'mlSCconv':ml_SCconv.magnitude,
    'muSCconv':mu_SCconv.magnitude,
    'sbPLconv':sb_PL_conv.magnitude,
    'mlPLconv':ml_PL_conv.magnitude,
    'muPLconv':mu_PL_conv.magnitude,

    'sfc1kmLapse':lapse0_1km,
    'sfc2kmLapse':lapse0_2km,
    'sfc3kmLapse':lapse0_3km,
    '3_6kmLapse':lapse3_6km,
    '5_700mbLapse':lapse5_700mb,
    'sbACBLlapse':sbACBLlapse.magnitude,
    'mlACBLlapse':mlACBLlapse.magnitude,
    'muACBLlapse':muACBLlapse.magnitude,
    'sb_lapse_LFC_LCL':sb_lapse_LFC_LCL,
    'ml_lapse_LFC_LCL':ml_lapse_LFC_LCL,
    'mu_lapse_LFC_LCL':mu_lapse_LFC_LCL,
    'esl_25_depth':esl_25_depth,
    'esl_40_depth':esl_40_depth,

    'sbMRD':sbMRD,
    'mlMRD':mlMRD,
    'muMRD':muMRD,
    'pwat':pwat,
    'ThetaEindex':tei,
    'maxTE3kmhgt':maxTE3kmhgt[0].magnitude,
    'maxQ3kmhgt':maxQ3kmhgt[0].magnitude,
    'meanRH100mb':meanRH100mb,
    'meanQ100mb':meanQ100mb,
    'meanTe100mb':meanTe100mb,
    'meanRH1km':meanRH1km,
    'meanQ1km':meanQ1km,
    'meanTe1km':meanTe1km,
    'meanRH150mb':meanRH150mb,
    'meanQ150mb':meanQ150mb,
    'meanTe150mb':meanTe150mb,
    'meanRH150_300mb':meanRH150_300mb,
    'meanQ150_300mb':meanQ150_300mb,
    'meanTe150_300mb':meanTe150_300mb,    
    'meanRHpbl':meanRHpbl,
    'meanQpbl':meanQpbl,
    'meanTepbl':meanTepbl,
    'meanRH1_3km':meanRH1_3km,
    'meanQ1_3km':meanQ1_3km,
    'meanTe1_3km':meanTe1_3km,    
    'meanRH_EIL':meanRH_EIL,
    'meanQ_EIL':meanQ_EIL,
    'meanTe_EIL':meanTe_EIL,    
    'sb_meanRH_LFC_LCL':sb_meanRH_LFC_LCL,
    'ml_meanRH_LFC_LCL':ml_meanRH_LFC_LCL,
    'mu_meanRH_LFC_LCL':mu_meanRH_LFC_LCL,
    'sb_meanQ_LFC_LCL':sb_meanQ_LFC_LCL,
    'ml_meanQ_LFC_LCL':ml_meanQ_LFC_LCL,
    'mu_meanQ_LFC_LCL':mu_meanQ_LFC_LCL,
    'sb_meanTe_LFC_LCL':sb_meanTe_LFC_LCL,
    'ml_meanTe_LFC_LCL':ml_meanTe_LFC_LCL,
    'mu_meanTe_LFC_LCL':mu_meanTe_LFC_LCL,
    'meanRHsb_ACBL':meanRHsb_ACBL,
    'meanRHml_ACBL':meanRHml_ACBL,
    'meanRHmu_ACBL':meanRHmu_ACBL,
    'meanQsb_ACBL':meanQsb_ACBL,
    'meanQml_ACBL':meanQml_ACBL,
    'meanQmu_ACBL':meanQmu_ACBL,
    'meanTesb_ACBL':meanTesb_ACBL,
    'meanTeml_ACBL':meanTeml_ACBL,
    'meanTemu_ACBL':meanTemu_ACBL,
    'meanRHsb_LCL2k':meanRHsb_LCL2k,
    'meanRHml_LCL2k':meanRHml_LCL2k,
    'meanRHmu_LCL2k':meanRHmu_LCL2k,
    'meanQsb_LCL2k':meanQsb_LCL2k,
    'meanQml_LCL2k':meanQml_LCL2k,
    'meanQmu_LCL2k':meanQmu_LCL2k,
    'meanTesb_LCL2k':meanTesb_LCL2k,
    'meanTeml_LCL2k':meanTeml_LCL2k,
    'meanTemu_LCL2k':meanTemu_LCL2k, 
    '500m_3km_RH':mean500m_3km_RH,
    '500m_4km_RH':mean500m_4km_RH,
    '500m_5km_RH':mean500m_5km_RH,
    '500m_6km_RH':mean500m_6km_RH,
    '1000m_3km_RH':mean1000m_3km_RH,
    '1000m_4km_RH':mean1000m_4km_RH,
    '1000m_5km_RH':mean1000m_5km_RH,
    '1000m_6km_RH':mean1000m_6km_RH,
    '1500m_3km_RH':mean1500m_3km_RH,
    '1500m_4km_RH':mean1500m_4km_RH,
    '1500m_5km_RH':mean1500m_5km_RH,
    '1500m_6km_RH':mean1500m_6km_RH,
    '2000m_3km_RH':mean2000m_3km_RH,
    '2000m_4km_RH':mean2000m_4km_RH,
    '2000m_5km_RH':mean2000m_5km_RH,
    '2000m_6km_RH':mean2000m_6km_RH,
    '2500m_3km_RH':mean2500m_3km_RH,
    '2500m_4km_RH':mean2500m_4km_RH,
    '2500m_5km_RH':mean2500m_5km_RH,
    '2500m_6km_RH':mean2500m_6km_RH,
    '3000m_4km_RH':mean3000m_4km_RH,
    '3000m_5km_RH':mean3000m_5km_RH,
    '3000m_6km_RH':mean3000m_6km_RH,

    'sbCAPE':sbcape,
    'mlCAPE':mlcape,
    'muCAPE':mucape,
    'sb_LCLCAPE':sb_LCLCAPE,
    'ml_LCLCAPE':ml_LCLCAPE,
    'mu_LCLCAPE':mu_LCLCAPE,
    'sb3CAPE':sb3kmCAPE,    
    'ml3CAPE':ml3kmCAPE,
    'mu3CAPE':mu3kmCAPE,
    'sb6CAPE':sb6kmCAPE,    
    'ml6CAPE':ml6kmCAPE,
    'mu6CAPE':mu6kmCAPE,
    'sbFrzCAPE':sbfrzCAPE,
    'mlFrzCAPE':mlfrzCAPE,
    'muFrzCAPE':mufrzCAPE,    
    'dCAPE':dCAPE[0],
    'sb_LImax':sb_limax,
    'ml_LImax':ml_limax,
    'mu_LImax':mu_limax,
    'eCAPE':ecape.magnitude,   
    'sbCAPE_500m_3km':sbCAPE_500m_3km,
    'sbCAPE_500m_4km':sbCAPE_500m_4km,
    'sbCAPE_500m_5km':sbCAPE_500m_5km,
    'sbCAPE_500m_6km':sbCAPE_500m_6km,
    'mlCAPE_500m_3km':mlCAPE_500m_3km,
    'mlCAPE_500m_4km':mlCAPE_500m_4km,
    'mlCAPE_500m_5km':mlCAPE_500m_5km,
    'mlCAPE_500m_6km':mlCAPE_500m_6km,
    'muCAPE_500m_3km':muCAPE_500m_3km,
    'muCAPE_500m_4km':muCAPE_500m_4km,
    'muCAPE_500m_5km':muCAPE_500m_5km,
    'muCAPE_500m_6km':muCAPE_500m_6km,
    'sbCAPE_1_3km':sbCAPE_1_3km,
    'sbCAPE_1_4km':sbCAPE_1_4km,
    'sbCAPE_1_5km':sbCAPE_1_5km,
    'sbCAPE_1_6km':sbCAPE_1_6km,
    'mlCAPE_1_3km':mlCAPE_1_3km,
    'mlCAPE_1_4km':mlCAPE_1_4km,
    'mlCAPE_1_5km':mlCAPE_1_5km,
    'mlCAPE_1_6km':mlCAPE_1_6km,
    'muCAPE_1_3km':muCAPE_1_3km,
    'muCAPE_1_4km':muCAPE_1_4km,
    'muCAPE_1_5km':muCAPE_1_5km,
    'muCAPE_1_6km':muCAPE_1_6km,
    'sbCAPE_1500m_3km':sbCAPE_1500m_3km,
    'sbCAPE_1500m_4km':sbCAPE_1500m_4km,
    'sbCAPE_1500m_5km':sbCAPE_1500m_5km,
    'sbCAPE_1500m_6km':sbCAPE_1500m_6km,
    'mlCAPE_1500m_3km':mlCAPE_1500m_3km,
    'mlCAPE_1500m_4km':mlCAPE_1500m_4km,
    'mlCAPE_1500m_5km':mlCAPE_1500m_5km,
    'mlCAPE_1500m_6km':mlCAPE_1500m_6km,
    'muCAPE_1500m_3km':muCAPE_1500m_3km,
    'muCAPE_1500m_4km':muCAPE_1500m_4km,
    'muCAPE_1500m_5km':muCAPE_1500m_5km,
    'muCAPE_1500m_6km':muCAPE_1500m_6km,
    'sbCAPE_2_3km':sbCAPE_2_3km,
    'sbCAPE_2_4km':sbCAPE_2_4km,
    'sbCAPE_2_5km':sbCAPE_2_5km,
    'sbCAPE_2_6km':sbCAPE_2_6km,
    'mlCAPE_2_3km':mlCAPE_2_3km,
    'mlCAPE_2_4km':mlCAPE_2_4km,
    'mlCAPE_2_5km':mlCAPE_2_5km,
    'mlCAPE_2_6km':mlCAPE_2_6km,
    'muCAPE_2_3km':muCAPE_2_3km,
    'muCAPE_2_4km':muCAPE_2_4km,
    'muCAPE_2_5km':muCAPE_2_5km,
    'muCAPE_2_6km':muCAPE_2_6km,    

    'sbCIN':sbcin,
    'mlCIN':mlcin,
    'muCIN':mucin,
    'ConvectiveT':ConvT,
    '700mbTemp':T_700,
    'sbCIN_500m_3km':sbCIN_500m_3km,
    'sbCIN_500m_4km':sbCIN_500m_4km,
    'sbCIN_500m_5km':sbCIN_500m_5km,
    'sbCIN_500m_6km':sbCIN_500m_6km,
    'mlCIN_500m_3km':mlCIN_500m_3km,
    'mlCIN_500m_4km':mlCIN_500m_4km,
    'mlCIN_500m_5km':mlCIN_500m_5km,
    'mlCIN_500m_6km':mlCIN_500m_6km,
    'muCIN_500m_3km':muCIN_500m_3km,
    'muCIN_500m_4km':muCIN_500m_4km,
    'muCIN_500m_5km':muCIN_500m_5km,
    'muCIN_500m_6km':muCIN_500m_6km,
    'sbCIN_1_3km':sbCIN_1_3km,
    'sbCIN_1_4km':sbCIN_1_4km,
    'sbCIN_1_5km':sbCIN_1_5km,
    'sbCIN_1_6km':sbCIN_1_6km,
    'mlCIN_1_3km':mlCIN_1_3km,
    'mlCIN_1_4km':mlCIN_1_4km,
    'mlCIN_1_5km':mlCIN_1_5km,
    'mlCIN_1_6km':mlCIN_1_6km,
    'muCIN_1_3km':muCIN_1_3km,
    'muCIN_1_4km':muCIN_1_4km,
    'muCIN_1_5km':muCIN_1_5km,
    'muCIN_1_6km':muCIN_1_6km,
    'sbCIN_1500m_3km':sbCIN_1500m_3km,
    'sbCIN_1500m_4km':sbCIN_1500m_4km,
    'sbCIN_1500m_5km':sbCIN_1500m_5km,
    'sbCIN_1500m_6km':sbCIN_1500m_6km,
    'mlCIN_1500m_3km':mlCIN_1500m_3km,
    'mlCIN_1500m_4km':mlCIN_1500m_4km,
    'mlCIN_1500m_5km':mlCIN_1500m_5km,
    'mlCIN_1500m_6km':mlCIN_1500m_6km,
    'muCIN_1500m_3km':muCIN_1500m_3km,
    'muCIN_1500m_4km':muCIN_1500m_4km,
    'muCIN_1500m_5km':muCIN_1500m_5km,
    'muCIN_1500m_6km':muCIN_1500m_6km,
    'sbCIN_2_3km':sbCIN_2_3km,
    'sbCIN_2_4km':sbCIN_2_4km,
    'sbCIN_2_5km':sbCIN_2_5km,
    'sbCIN_2_6km':sbCIN_2_6km,
    'mlCIN_2_3km':mlCIN_2_3km,
    'mlCIN_2_4km':mlCIN_2_4km,
    'mlCIN_2_5km':mlCIN_2_5km,
    'mlCIN_2_6km':mlCIN_2_6km,
    'muCIN_2_3km':muCIN_2_3km,
    'muCIN_2_4km':muCIN_2_4km,
    'muCIN_2_5km':muCIN_2_5km,
    'muCIN_2_6km':muCIN_2_6km,

    'bunkersRMU':bunkersU,
    'bunkersRMV':bunkersV,
    'bunkersRM':vectorMag,
    'bunkersRMangle':angle,

    'sfc500mSRH':srh500m,
    'sfc1kmSRH':srh1km,
    'sfc3kmSRH':srh3km,
    'ESRH':effective_srh,

    '0_500_m_BWD':sfc500mshear,
    '0_1_km_BWD':sfc1shear,
    '0_2_km_BWD':sfc2shear,
    '0_3_km_BWD':sfc3shear,
    '0_5_km_BWD':sfc5shear,
    '0_6_km_BWD':sfc6shear,
    '0_8_km_BWD':sfc8shear,
    'EIL_BWD':EILshear,
    'sb_Effective_BWD':sb_ebwspd,
    'ml_Effective_BWD':ml_ebwspd,
    'mu_Effective_BWD':mu_ebwspd,
    'sb_LCL_EL_BWD':sb_lclELshear,
    'ml_LCL_EL_BWD':ml_lclELshear,
    'mu_LCL_EL_BWD':mu_lclELshear,   

    '0_500mSRflow':srflow0500m,
    '0_1kmSRflow':srflow01km,
    '0_2kmSRflow':srflow02km,
    '0_3kmSRflow':srflow03km,
    '0_6kmSRflow':srflow06km,
    '0_8kmSRflow':srflow08km,
    '4_5kmSRflow':srflow4_5km,
    '4_6kmSRflow':srflow4_6km,
    'sb_LCL_EL_SRflow':sb_srflowLCL_EL,
    'ml_LCL_EL_SRflow':ml_srflowLCL_EL,
    'mu_LCL_EL_SRflow':mu_srflowLCL_EL,    
    'EIL_SRflow':srflowEIL,

    'sbSCshear':sb_SC_shear,
    'mlSCshear':ml_SC_shear,
    'muSCshear':mu_SC_shear,
    'sbACBLshear':sb_acbl_shear,
    'mlACBLshear':ml_acbl_shear,
    'muACBLshear':mu_acbl_shear,
    'critAngle':critAng,
    'sb_brnSHR':sb_brnSHR,
    'ml_brnSHR':ml_brnSHR,
    'mu_brnSHR':mu_brnSHR,
    'MeanShearValue_500m':MeanShearValue_500m,
    'MeanSWvalue_500m':MeanSWvalue_500m,
    'MeanSWpercvalue_500m':MeanSWpercvalue_500m,
    'MeanShearValue_1km':MeanShearValue_1km,
    'MeanSWvalue_1km':MeanSWvalue_1km,
    'MeanSWpercvalue_1km':MeanSWpercvalue_1km,
    'MeanShearValue_3km':MeanShearValue_3km,
    'MeanSWvalue_3km':MeanSWvalue_3km,
    'MeanSWpercvalue_3km':MeanSWpercvalue_3km,
    '1_3km_Shear':MeanShearValue1_3km,
    '1_3km_SW':MeanSWvalue1_3km,
    '1_3km_SW_perc':MeanSWpercvalue1_3km,    

    'SCP':SCP,
    'STP_FL':STP_FL,
    'STP_EL':STP_EL,
    'STP_500':STP_500,
    'VTP':VTP,
    'sbBRN':sbBRN,
    'mlBRN':mlBRN,
    'muBRN':muBRN,
    'DCP':dcp,
    'WDP':wndg,
    'SHP':ship,
    'SigSev':sig_sev,
    'sb01ehi':sb_0_1ehi,
    'sb03ehi':sb_0_3ehi,
    'ml01ehi':ml_0_1ehi,
    'ml03ehi':ml_0_3ehi,
    'mu01ehi':mu_0_1ehi,
    'mu03ehi':mu_0_3ehi
    }
    
    for p in range(1):
        df1.loc[len(df1)]=new_row
        
    #np.savetxt(r'C:\Users\Kyled\Desktop\UNL MS Thesis\6 - SkewTs\Soundings\HRRR_'+str(prof_num)+'_'+str(StormID)+'_'+str(hour)+'_'+str(lat[x][y])+'_'+str(lon[x][y])+'.txt', dfSounding2,fmt='%1.9f')
    
    print(i,sbcape)
    i+=1

C:\Users\kyled\AppData\Local\Temp\ipykernel_170976\2680908632.py:87: UserWarning: Relative humidity >120%, ensure proper units.
  Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)
D:\Anaconda3\envs\THESIS\lib\site-packages\metpy\calc\thermo.py:1396: RuntimeWarning: divide by zero encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
D:\Anaconda3\envs\THESIS\lib\site-packages\metpy\calc\thermo.py:1397: RuntimeWarning: invalid value encountered in divide
  return mpconsts.nounit.zero_degc + 243.5 * val / (17.67 - val)


34.01502 -98.82429
34.01524079477599 -98.83613665918432 



D:\Anaconda3\envs\THESIS\lib\site-packages\numpy\lib\function_base.py:1594: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)
D:\Anaconda3\envs\THESIS\lib\site-packages\sharppy\sharptab\params.py:1310: RuntimeWarning: invalid value encountered in double_scalars
  return (tv2 - tv1) / (z2 - z1) * -1000.


0 4193.3277570274195


C:\Users\kyled\AppData\Local\Temp\ipykernel_170976\2680908632.py:87: UserWarning: Relative humidity >120%, ensure proper units.
  Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)
D:\Anaconda3\envs\THESIS\lib\site-packages\metpy\calc\thermo.py:1396: RuntimeWarning: divide by zero encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
D:\Anaconda3\envs\THESIS\lib\site-packages\metpy\calc\thermo.py:1397: RuntimeWarning: invalid value encountered in divide
  return mpconsts.nounit.zero_degc + 243.5 * val / (17.67 - val)


...

208 2605.994283715912


C:\Users\kyled\AppData\Local\Temp\ipykernel_170976\2680908632.py:87: UserWarning: Relative humidity >120%, ensure proper units.
  Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)
D:\Anaconda3\envs\THESIS\lib\site-packages\metpy\calc\thermo.py:1396: RuntimeWarning: divide by zero encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
D:\Anaconda3\envs\THESIS\lib\site-packages\metpy\calc\thermo.py:1397: RuntimeWarning: invalid value encountered in divide
  return mpconsts.nounit.zero_degc + 243.5 * val / (17.67 - val)


34.73881 -97.16493
34.74939434706999 -97.17802534849926 



D:\Anaconda3\envs\THESIS\lib\site-packages\numpy\lib\function_base.py:1594: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)


209 4030.630364752632


In [131]:
output_path = 'D:/THESIS/__UNL MS Thesis (MAIN)__/3 - Obtain Vertical Profiles/5-22_NearInflowParameters.csv'  # Specify your desired file path and name
df1.to_csv(output_path, index=False)